In [1]:
#Load all csv
import pandas as pd
import numpy as np

number_of_testers = 34
filenames = ["SL_LIT", "SL_BIG", "FA_LIT", "FA_BIG"]
animname = ["VB", "HS"]
all_dfs = {}

for tester in range(1, number_of_testers+1):
    for session in range(1, 4):
        for trial in range(1, 4):
            for fname in filenames:
                for anim_name in animname:
                    key = f"T{tester}_S{session}_TRY{trial}_{anim_name}_{fname}"
                    path = rf"C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Results_csv\Tester{tester}\Session{session}\Trial{trial}\T{tester}-S{session}-TRY{trial}-{anim_name}_{fname}.csv"
                    try:
                        df = pd.read_csv(path)
                        all_dfs[key] = df
                        print(f"Loaded: {key}")
                    except FileNotFoundError:
                        print(f"Missing: {key}")


Loaded: T1_S1_TRY1_VB_SL_LIT
Loaded: T1_S1_TRY1_HS_SL_LIT
Loaded: T1_S1_TRY1_VB_SL_BIG
Loaded: T1_S1_TRY1_HS_SL_BIG
Loaded: T1_S1_TRY1_VB_FA_LIT
Loaded: T1_S1_TRY1_HS_FA_LIT
Loaded: T1_S1_TRY1_VB_FA_BIG
Loaded: T1_S1_TRY1_HS_FA_BIG
Loaded: T1_S1_TRY2_VB_SL_LIT
Loaded: T1_S1_TRY2_HS_SL_LIT
Loaded: T1_S1_TRY2_VB_SL_BIG
Loaded: T1_S1_TRY2_HS_SL_BIG
Loaded: T1_S1_TRY2_VB_FA_LIT
Loaded: T1_S1_TRY2_HS_FA_LIT
Loaded: T1_S1_TRY2_VB_FA_BIG
Loaded: T1_S1_TRY2_HS_FA_BIG
Loaded: T1_S1_TRY3_VB_SL_LIT
Loaded: T1_S1_TRY3_HS_SL_LIT
Loaded: T1_S1_TRY3_VB_SL_BIG
Loaded: T1_S1_TRY3_HS_SL_BIG
Loaded: T1_S1_TRY3_VB_FA_LIT
Loaded: T1_S1_TRY3_HS_FA_LIT
Loaded: T1_S1_TRY3_VB_FA_BIG
Loaded: T1_S1_TRY3_HS_FA_BIG
Loaded: T1_S2_TRY1_VB_SL_LIT
Loaded: T1_S2_TRY1_HS_SL_LIT
Loaded: T1_S2_TRY1_VB_SL_BIG
Loaded: T1_S2_TRY1_HS_SL_BIG
Loaded: T1_S2_TRY1_VB_FA_LIT
Loaded: T1_S2_TRY1_HS_FA_LIT
Loaded: T1_S2_TRY1_VB_FA_BIG
Loaded: T1_S2_TRY1_HS_FA_BIG
Loaded: T1_S2_TRY2_VB_SL_LIT
Loaded: T1_S2_TRY2_HS_SL_LIT
Loaded: T1_S2_

In [2]:
#Fixation vector for each file
import duckdb

fpogs_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    
    result = duckdb.query("""
        SELECT 
            AVG(FPOGX) AS FPOGX, 
            AVG(FPOGY) AS FPOGY, 
            MAX(FPOGS) AS FPOGS, 
            MAX(FPOGD) AS FPOGD, 
            FPOGID
        FROM df
        WHERE FPOGV = '1'
        GROUP BY FPOGID
        ORDER BY FPOGID
    """).to_df()
    
    # Extract vectors from result
    fpogx_vector = result['FPOGX'].tolist()
    fpogy_vector = result['FPOGY'].tolist()
    fpogs_values = result['FPOGS'].tolist()
    fpogd_vector = result['FPOGD'].tolist()
    fpogid_vector = result['FPOGID'].tolist()
    
    # Append structured result
    fpogs_vector.append({
        'file_key': key,
        'FPOGX': fpogx_vector,
        'FPOGY': fpogy_vector,
        'FPOGS': fpogs_values,
        'FPOGD': fpogd_vector,
        'FPOGID': fpogid_vector
    })

# Convert to DataFrame
fpogs_vector_df = pd.DataFrame(fpogs_vector)

# Display the DataFrame
display(fpogs_vector_df)


,file_key,FPOGX,FPOGY,FPOGS,FPOGD,FPOGID
0,T1_S1_TRY1_VB_SL_LIT,"[0.5115149999999999, 0.6082785, 0.609104, 0.41...","[-0.26439, 0.2826875, 0.2863204444444445, 0.93...","[1025.16162, 1027.95508, 1028.25623, 1028.6210...","[0.04688, 0.29431, 0.31482, 0.11023, 0.1748, 0...","[2139, 2147, 2148, 2149, 2150, 2151, 2152, 215..."
1,T1_S1_TRY1_HS_SL_LIT,"[0.61804, 0.6169896491228072, 1.17444795918367...","[0.27416, 0.28703596491228067, 0.4451561224489...","[1099.13293, 1099.75549, 1100.20422, 1100.5535...","[0.61572, 0.40222, 0.34204, 0.18689, 0.18347, ...","[2446, 2447, 2448, 2449, 2450, 2451, 2452, 245..."
2,T1_S1_TRY1_VB_SL_BIG,"[0.6122326388888888, 0.4873268421052632, 0.188...","[0.26845097222222236, 0.943437894736842, 0.915...","[1117.20703, 1118.19336, 1118.35742, 1118.6656...","[0.93579, 0.1377, 0.28149, 0.22046, 0.26172, 0...","[2499, 2500, 2501, 2502, 2503, 2504, 2505, 250..."
3,T1_S1_TRY1_HS_SL_BIG,"[0.6195031481481484, 0.6214363636363637, 1.171...","[0.24344759259259255, 0.22857272727272732, 0.4...","[1045.63892, 1046.37512, 1046.50952, 1046.7376...","[0.72949, 0.08789, 0.22131, 0.20044, 0.1947, 0...","[2207, 2208, 2209, 2210, 2211, 2212, 2213, 221..."
4,T1_S1_TRY1_VB_FA_LIT,"[0.6168279999999999, 0.6221484782608693, 0.486...","[0.23235699999999998, 0.24510369565217388, 0.8...","[1062.7196, 1064.00647, 1064.37427, 1064.52197...","[1.28003, 0.32092, 0.12061, 0.20105, 0.24524, ...","[2305, 2306, 2307, 2308, 2309, 2310, 2311, 231..."
...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[0.6140952941176472, 1.2235079545454544, 1.200...","[0.13683985294117648, 0.2810161363636365, 0.29...","[513.41589, 515.30408, 515.63226, 516.36237, 5...","[1.84009, 0.32172, 0.72406, 0.12732, 0.18842, ...","[1812, 1813, 1814, 1815, 1816, 1817, 1818, 181..."
2372,T34_S1_TRY3_VB_FA_LIT,"[0.6191450000000001, 0.6202076785714287, 0.506...","[0.14264, 0.15240321428571435, 0.7690688235294...","[494.80014, 495.82553, 496.40778, 496.56259, 4...","[0.27597, 0.53595, 0.12921, 0.30081, 0.3483, 0...","[1734, 1735, 1736, 1737, 1738, 1739, 1740, 174..."
2373,T34_S1_TRY3_HS_FA_LIT,"[0.6175371153846154, 1.2112063636363635, 1.188...","[0.13248326923076925, 0.2696522727272727, 0.27...","[626.63928, 628.09259, 628.41333, 628.97729, 6...","[1.40656, 0.3111, 0.55743, 0.27435, 0.15442, 0...","[2225, 2226, 2227, 2228, 2229, 2230, 2231, 223..."
2374,T34_S1_TRY3_VB_FA_BIG,"[0.6155355208333333, 0.23220624999999995, 0.38...","[0.14507489583333333, 0.6275642499999999, 0.60...","[533.45734, 534.32819, 534.63684, 534.8302, 53...","[0.8241, 0.28174, 0.16711, 0.16791, 0.23468, 0...","[1871, 1872, 1873, 1874, 1875, 1876, 1877, 187..."


In [3]:
#Show the fixation data in a nicer way into a csv file
import pandas as pd

# Flatten the list into a structured tabular format
flattened_fpogs_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    for fpogid, fpogx, fpogy, fpogs, fpogd in zip(
        item['FPOGID'], item['FPOGX'], item['FPOGY'], item['FPOGS'], item['FPOGD']
    ):
        flattened_fpogs_data.append({
            'file_key': file_key,
            'FPOGID': fpogid,
            'FPOGX': fpogx,
            'FPOGY': fpogy,
            'FPOGS': fpogs,
            'FPOGD': fpogd
        })

# Create the final DataFrame
fpogs_table_df = pd.DataFrame(flattened_fpogs_data)

# Display the table
display(fpogs_table_df)

#Create the csv 
fpogs_table_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\fixation_vector.csv', index=False)


,file_key,FPOGID,FPOGX,FPOGY,FPOGS,FPOGD
0,T1_S1_TRY1_VB_SL_LIT,2139,0.511515,-0.264390,1025.16162,0.04688
1,T1_S1_TRY1_VB_SL_LIT,2147,0.608279,0.282687,1027.95508,0.29431
2,T1_S1_TRY1_VB_SL_LIT,2148,0.609104,0.286320,1028.25623,0.31482
3,T1_S1_TRY1_VB_SL_LIT,2149,0.415205,0.930494,1028.62109,0.11023
4,T1_S1_TRY1_VB_SL_LIT,2150,0.219798,0.931156,1028.75842,0.17480
...,...,...,...,...,...,...
140688,T34_S1_TRY3_HS_FA_BIG,2016,1.210023,0.292787,566.11884,0.40851
140689,T34_S1_TRY3_HS_FA_BIG,2017,1.197350,0.260016,566.53394,0.14789
140690,T34_S1_TRY3_HS_FA_BIG,2018,1.208001,0.294235,566.68921,0.22009
140691,T34_S1_TRY3_HS_FA_BIG,2019,1.206055,0.294030,566.91620,0.30841


In [4]:
#Create the dataframe with the first column, which is the number of fixations
feature_vectors = []
# Loop through all files in the dataframe
for key, df in all_dfs.items():
    #Start from 1 because the fixation with id zero is our first fixation
    num_fix = 1
    if 'FPOGID' in df.columns and not df.empty:
        # Convert to list for easier iteration
        fix_id_series = df['FPOGID'].tolist() 
        # Check that the list is not empty
        if fix_id_series: 
            previous_id = fix_id_series[0]
        # Start from the second element
        for current_id in fix_id_series[1:]:
            if current_id != previous_id:
                num_fix += 1
                previous_id = current_id

    # Create a feature vector with key and features
    feature_vectors.append({
        'file_key': key,
        'f0': num_fix
    })
     
# Create a DataFrame from all feature vectors
feature_df = pd.DataFrame(feature_vectors)

# Display the current dataset
display(feature_df)


,file_key,f0
0,T1_S1_TRY1_VB_SL_LIT,57
1,T1_S1_TRY1_HS_SL_LIT,45
2,T1_S1_TRY1_VB_SL_BIG,59
3,T1_S1_TRY1_HS_SL_BIG,90
4,T1_S1_TRY1_VB_FA_LIT,65
...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48
2372,T34_S1_TRY3_VB_FA_LIT,71
2373,T34_S1_TRY3_HS_FA_LIT,58
2374,T34_S1_TRY3_VB_FA_BIG,78


In [5]:
#Obtain the duration of every FPOGID in a file
import duckdb

fix_duration_vector = []

for fix in fpogs_vector:
    
    fpogd_vector = fix['FPOGD']
    fpogid_vector = fix['FPOGID']

    # Store in the feature vector list
    fix_duration_vector.append({
        'file_key': key,
        'FPOGID': fpogid_vector,
        'FPOGD': fpogd_vector
    })

# Create a final DataFrame
fix_duration_vector_df = pd.DataFrame(fix_duration_vector)

# Display the DataFrame
display(fix_duration_vector_df)


,file_key,FPOGID,FPOGD
0,T34_S1_TRY3_HS_FA_BIG,"[2139, 2147, 2148, 2149, 2150, 2151, 2152, 215...","[0.04688, 0.29431, 0.31482, 0.11023, 0.1748, 0..."
1,T34_S1_TRY3_HS_FA_BIG,"[2446, 2447, 2448, 2449, 2450, 2451, 2452, 245...","[0.61572, 0.40222, 0.34204, 0.18689, 0.18347, ..."
2,T34_S1_TRY3_HS_FA_BIG,"[2499, 2500, 2501, 2502, 2503, 2504, 2505, 250...","[0.93579, 0.1377, 0.28149, 0.22046, 0.26172, 0..."
3,T34_S1_TRY3_HS_FA_BIG,"[2207, 2208, 2209, 2210, 2211, 2212, 2213, 221...","[0.72949, 0.08789, 0.22131, 0.20044, 0.1947, 0..."
4,T34_S1_TRY3_HS_FA_BIG,"[2305, 2306, 2307, 2308, 2309, 2310, 2311, 231...","[1.28003, 0.32092, 0.12061, 0.20105, 0.24524, ..."
...,...,...,...
2371,T34_S1_TRY3_HS_FA_BIG,"[1812, 1813, 1814, 1815, 1816, 1817, 1818, 181...","[1.84009, 0.32172, 0.72406, 0.12732, 0.18842, ..."
2372,T34_S1_TRY3_HS_FA_BIG,"[1734, 1735, 1736, 1737, 1738, 1739, 1740, 174...","[0.27597, 0.53595, 0.12921, 0.30081, 0.3483, 0..."
2373,T34_S1_TRY3_HS_FA_BIG,"[2225, 2226, 2227, 2228, 2229, 2230, 2231, 223...","[1.40656, 0.3111, 0.55743, 0.27435, 0.15442, 0..."
2374,T34_S1_TRY3_HS_FA_BIG,"[1871, 1872, 1873, 1874, 1875, 1876, 1877, 187...","[0.8241, 0.28174, 0.16711, 0.16791, 0.23468, 0..."


In [6]:
#Minimum fixation duration

min_fix_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    fpogd_list = item['FPOGD']
    
    if fpogd_list:
        min_fpogd = np.min(fpogd_list)
    else:
        min_fpogd = None  
    
    min_fix_data.append({ 
        'file_key': file_key,
        'f1': min_fpogd
    })

# Convert to DataFrame for easy use or merging
min_fixation_df = pd.DataFrame(min_fix_data)

feature_df = feature_df.merge(min_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087
...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292


In [7]:
#Maximum fixation duration that join with the feature vector
max_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        max_duration = max(max_fpogd)
    else:
        max_duration = None 

    max_fixation_data.append({
        'file_key': file_key,
        'f2': max_duration
    })

# Create pandas DataFrame
max_fixation_df = pd.DataFrame(max_fixation_data)

feature_df = feature_df.merge(max_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003
...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410


In [8]:
#Arithmetic mean of fixation duration that join with feature vector
mean_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        mean_duration = np.mean(max_fpogd)
    else:
        mean_duration = None 

    mean_fixation_data.append({
        'file_key': file_key,
        'f3': mean_duration
    })

# Create pandas DataFrame
mean_fixation_df = pd.DataFrame(mean_fixation_data)

feature_df = feature_df.merge(mean_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979
...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235


In [9]:
#Geometric mean of fixation duration that join with feature vector
from scipy.stats import gmean

geom_mean_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        geom_mean_duration = gmean(max_fpogd)
    else:
        geom_mean_duration = None 

    geom_mean_fixation_data.append({
        'file_key': file_key,
        'f4': geom_mean_duration
    })

# Create pandas DataFrame
geom_mean_fixation_df = pd.DataFrame(geom_mean_fixation_data)

feature_df = feature_df.merge(geom_mean_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705
...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477


In [10]:
#Median of fixation duration that join with feature vector
median_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        median_duration = np.median(max_fpogd)
    else:
        median_duration = None 

    median_fixation_data.append({
        'file_key': file_key,
        'f5': median_duration
    })

# Create pandas DataFrame
median_fixation_df = pd.DataFrame(median_fixation_data)

feature_df = feature_df.merge(median_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119
...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089


In [11]:
#Standard deviation of fixation duration that join with feature vector
std_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        std_duration = np.std(max_fpogd)
    else:
        std_duration = None 

    std_fixation_data.append({
        'file_key': file_key,
        'f6': std_duration
    })

# Create pandas DataFrame
std_fixation_df = pd.DataFrame(std_fixation_data)

feature_df = feature_df.merge(std_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702
...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828


In [12]:
#Median absolute deviation of fixation duration that join with feature vector
from scipy.stats import median_abs_deviation

mad_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        mad_duration = median_abs_deviation(max_fpogd, scale=1)
    else:
        mad_duration = None 

    mad_fixation_data.append({
        'file_key': file_key,
        'f7': mad_duration
    })

# Create pandas DataFrame
mad_fixation_df = pd.DataFrame(mad_fixation_data)

feature_df = feature_df.merge(mad_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050
...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150


In [13]:
#Skewness of fixation duration that join with feature vector
from scipy.stats import skew

skew_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        skew_duration = skew(max_fpogd)
    else:
        skew_duration = None 

    skew_fixation_data.append({
        'file_key': file_key,
        'f8': skew_duration
    })

# Create pandas DataFrame
skew_fixation_df = pd.DataFrame(skew_fixation_data)

feature_df = feature_df.merge(skew_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584
...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048


In [14]:
#Interquartile range of fixation duration that join with feature vector
from scipy.stats import iqr

iqr_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        iqr_duration = iqr(max_fpogd)
    else:
        iqr_duration = None 

    iqr_fixation_data.append({
        'file_key': file_key,
        'f9': iqr_duration
    })

# Create pandas DataFrame
iqr_fixation_df = pd.DataFrame(iqr_fixation_data)

feature_df = feature_df.merge(iqr_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827
...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912


In [15]:
#Kurtosis of fixation duration that join with feature vector
from scipy.stats import kurtosis

kurt_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        kurt_duration = kurtosis(max_fpogd)
    else:
        kurt_duration = None 

    kurt_fixation_data.append({
        'file_key': file_key,
        'f10': kurt_duration
    })

# Create pandas DataFrame
kurt_fixation_df = pd.DataFrame(kurt_fixation_data)

feature_df = feature_df.merge(kurt_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846
...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832


In [16]:
#Distance between consecutive fixations
#The distance is computed taking in consideration the euclidean distance from the average FPOGX and FPOGY of two consecutive fixations
from scipy.spatial import distance

fpogs_diff_vector = []

for fix in fpogs_vector:
    file_key = fix['file_key']
    fpogid_vector = fix['FPOGID']
    fpogx_vector = fix['FPOGX']
    fpogy_vector = fix['FPOGY']

    # Build list of consecutive FPOGID pairs 
    fpogid_pairs = []
    for i in range(len(fpogid_vector) - 1):
        pair = (fpogid_vector[i], fpogid_vector[i + 1])
        fpogid_pairs.append(pair)

    # Build list of consecutive Euclidean distances
    fpogs_differences = []
    for i in range(len(fpogx_vector) - 1):
            point1 = [fpogx_vector[i], fpogy_vector[i]]
            point2 = [fpogx_vector[i + 1], fpogy_vector[i + 1]]
            dist = distance.euclidean(point1, point2)
            fpogs_differences.append(dist)
       
    # Append result 
    fpogs_diff_vector.append({
        'file_key': file_key,
        'FPOGID_pairs': fpogid_pairs,
        'FPOGS_differences': fpogs_differences
    })

# Create the DataFrame
fpogs_diff_vector_df = pd.DataFrame(fpogs_diff_vector)

# Display the DataFrame
display(fpogs_diff_vector_df)


,file_key,FPOGID_pairs,FPOGS_differences
0,T1_S1_TRY1_VB_SL_LIT,"[(2139, 2147), (2147, 2148), (2148, 2149), (21...","[0.5555690469586116, 0.003725551715708707, 0.6..."
1,T1_S1_TRY1_HS_SL_LIT,"[(2446, 2447), (2447, 2448), (2448, 2449), (24...","[0.012918734821471583, 0.5794495247002519, 0.0..."
2,T1_S1_TRY1_VB_SL_BIG,"[(2499, 2500), (2500, 2501), (2501, 2502), (25...","[0.6864465045696638, 0.2998852819829497, 0.155..."
3,T1_S1_TRY1_HS_SL_BIG,"[(2207, 2208), (2208, 2209), (2209, 2210), (22...","[0.01499996468022524, 0.5843035217134482, 0.04..."
4,T1_S1_TRY1_VB_FA_LIT,"[(2305, 2306), (2306, 2307), (2307, 2308), (23...","[0.013812521094048411, 0.6056363728885648, 0.2..."
...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[(1812, 1813), (1813, 1814), (1814, 1815), (18...","[0.6262352524341083, 0.028195674117363706, 0.0..."
2372,T34_S1_TRY3_VB_FA_LIT,"[(1734, 1735), (1735, 1736), (1736, 1737), (17...","[0.009820877706954228, 0.6270172140925853, 0.3..."
2373,T34_S1_TRY3_HS_FA_LIT,"[(2225, 2226), (2226, 2227), (2227, 2228), (22...","[0.6093098652081873, 0.02469854153313105, 0.00..."
2374,T34_S1_TRY3_VB_FA_BIG,"[(1871, 1872), (1872, 1873), (1873, 1874), (18...","[0.6162282911079157, 0.1518662078227957, 0.135..."


In [17]:
#Minimum difference between consecutive fixations that join with feature vector
min_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        min_diff = np.min(differences)
    else:
        min_diff = None

    min_diff_data.append({
        'file_key': file_key,
        'f11': min_diff
    })

# Create DataFrame from min differences
min_diff_df = pd.DataFrame(min_diff_data)

feature_df = feature_df.merge(min_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151


In [18]:
#Maximum difference between consecutive fixations that join with feature vector
max_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        max_diff = np.max(differences)
    else:
        max_diff = None

    max_diff_data.append({
        'file_key': file_key,
        'f12': max_diff
    })

# Create the DataFrame 
max_diff_df = pd.DataFrame(max_diff_data)

feature_df = feature_df.merge(max_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726,0.791343
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038,0.579450
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712,0.800112
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676,0.584304
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813,0.784073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312,0.626235
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821,0.767095
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254,0.609310
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151,0.718932


In [19]:
#Arithmetic mean between consecutive fixations that join with feature vector
mean_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        mean_diff = np.mean(differences)
    else:
        mean_diff = None

    mean_diff_data.append({
        'file_key': file_key,
        'f13': mean_diff
    })

# Create the DataFrame 
mean_diff_df = pd.DataFrame(mean_diff_data)

feature_df = feature_df.merge(mean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726,0.791343,0.130331
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038,0.579450,0.044324
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712,0.800112,0.196228
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676,0.584304,0.093602
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813,0.784073,0.140735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312,0.626235,0.047437
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821,0.767095,0.183198
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254,0.609310,0.047033
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151,0.718932,0.231294


In [20]:
#Geometric mean between consecutive fixations that join with feature vector
gmean_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        gmean_diff = gmean(differences)
    else:
        gmean_diff = None

    gmean_diff_data.append({
        'file_key': file_key,
        'f14': gmean_diff
    })

# Create the DataFrame 
gmean_diff_df = pd.DataFrame(gmean_diff_data)

feature_df = feature_df.merge(gmean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726,0.791343,0.130331,0.076023
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038,0.579450,0.044324,0.029852
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712,0.800112,0.196228,0.136460
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676,0.584304,0.093602,0.068154
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813,0.784073,0.140735,0.089522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312,0.626235,0.047437,0.032359
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821,0.767095,0.183198,0.121121
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254,0.609310,0.047033,0.032646
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151,0.718932,0.231294,0.158096


In [21]:
#Median between consecutive fixations that join with feature vector
median_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        median_diff = np.median(differences)
    else:
        median_diff = None

    median_diff_data.append({
        'file_key': file_key,
        'f15': median_diff
    })

# Create the DataFrame 
median_diff_df = pd.DataFrame(median_diff_data)

feature_df = feature_df.merge(median_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726,0.791343,0.130331,0.076023,0.077163
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038,0.579450,0.044324,0.029852,0.028986
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712,0.800112,0.196228,0.136460,0.119380
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676,0.584304,0.093602,0.068154,0.083411
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813,0.784073,0.140735,0.089522,0.080418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312,0.626235,0.047437,0.032359,0.033660
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821,0.767095,0.183198,0.121121,0.109994
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254,0.609310,0.047033,0.032646,0.035231
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151,0.718932,0.231294,0.158096,0.170818


In [22]:
#STD between consecutive fixations that join with feature vector
std_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        std_diff = np.std(differences)
    else:
        std_diff = None

    std_diff_data.append({
        'file_key': file_key,
        'f16': std_diff
    })

# Create the DataFrame 
std_diff_df = pd.DataFrame(std_diff_data)

feature_df = feature_df.merge(std_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726,0.791343,0.130331,0.076023,0.077163,0.188463
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038,0.579450,0.044324,0.029852,0.028986,0.082984
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712,0.800112,0.196228,0.136460,0.119380,0.201262
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676,0.584304,0.093602,0.068154,0.083411,0.077927
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813,0.784073,0.140735,0.089522,0.080418,0.183063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312,0.626235,0.047437,0.032359,0.033660,0.087031
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821,0.767095,0.183198,0.121121,0.109994,0.190756
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254,0.609310,0.047033,0.032646,0.035231,0.077546
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151,0.718932,0.231294,0.158096,0.170818,0.190197


In [23]:
#MAD between consecutive fixations that join with feature vector
mad_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        mad_diff = median_abs_deviation(differences, scale=1)
    else:
        mad_diff = None

    mad_diff_data.append({
        'file_key': file_key,
        'f17': mad_diff
    })

# Create the DataFrame 
mad_diff_df = pd.DataFrame(mad_diff_data)

feature_df = feature_df.merge(mad_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726,0.791343,0.130331,0.076023,0.077163,0.188463,0.029011
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038,0.579450,0.044324,0.029852,0.028986,0.082984,0.008911
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712,0.800112,0.196228,0.136460,0.119380,0.201262,0.044493
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676,0.584304,0.093602,0.068154,0.083411,0.077927,0.045900
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813,0.784073,0.140735,0.089522,0.080418,0.183063,0.032064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312,0.626235,0.047437,0.032359,0.033660,0.087031,0.013811
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821,0.767095,0.183198,0.121121,0.109994,0.190756,0.053279
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254,0.609310,0.047033,0.032646,0.035231,0.077546,0.015513
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151,0.718932,0.231294,0.158096,0.170818,0.190197,0.094587


In [24]:
#Skewness between consecutive fixations that join with feature vector
skew_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        skew_diff = skew(differences)
    else:
        skew_diff = None

    skew_diff_data.append({
        'file_key': file_key,
        'f18': skew_diff
    })

# Create the DataFrame 
skew_diff_df = pd.DataFrame(skew_diff_data)

feature_df = feature_df.merge(skew_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,0.133180,0.970100,0.003726,0.791343,0.130331,0.076023,0.077163,0.188463,0.029011,2.811712
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,0.261470,-0.020872,0.004038,0.579450,0.044324,0.029852,0.028986,0.082984,0.008911,6.079763
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,0.080200,14.786590,0.035712,0.800112,0.196228,0.136460,0.119380,0.201262,0.044493,1.943941
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,0.060790,11.841482,0.010676,0.584304,0.093602,0.068154,0.083411,0.077927,0.045900,2.926825
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,0.125827,25.476846,0.013813,0.784073,0.140735,0.089522,0.080418,0.183063,0.032064,2.735644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,0.193243,15.822441,0.006312,0.626235,0.047437,0.032359,0.033660,0.087031,0.013811,6.243230
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,0.114255,2.267225,0.009821,0.767095,0.183198,0.121121,0.109994,0.190756,0.053279,1.878249
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,0.187945,17.331336,0.003254,0.609310,0.047033,0.032646,0.035231,0.077546,0.015513,6.668649
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,0.110912,23.346832,0.008151,0.718932,0.231294,0.158096,0.170818,0.190197,0.094587,1.058302


In [25]:
#Interquartile range between consecutive fixations that join with feature vector
iqr_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        iqr_diff = iqr(differences)
    else:
        iqr_diff = None

    iqr_diff_data.append({
        'file_key': file_key,
        'f19': iqr_diff
    })

# Create the DataFrame 
iqr_diff_df = pd.DataFrame(iqr_diff_data)

feature_df = feature_df.merge(iqr_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.970100,0.003726,0.791343,0.130331,0.076023,0.077163,0.188463,0.029011,2.811712,0.056973
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,-0.020872,0.004038,0.579450,0.044324,0.029852,0.028986,0.082984,0.008911,6.079763,0.020567
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,14.786590,0.035712,0.800112,0.196228,0.136460,0.119380,0.201262,0.044493,1.943941,0.107858
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,11.841482,0.010676,0.584304,0.093602,0.068154,0.083411,0.077927,0.045900,2.926825,0.092732
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,25.476846,0.013813,0.784073,0.140735,0.089522,0.080418,0.183063,0.032064,2.735644,0.070823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,15.822441,0.006312,0.626235,0.047437,0.032359,0.033660,0.087031,0.013811,6.243230,0.026607
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,2.267225,0.009821,0.767095,0.183198,0.121121,0.109994,0.190756,0.053279,1.878249,0.132005
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,17.331336,0.003254,0.609310,0.047033,0.032646,0.035231,0.077546,0.015513,6.668649,0.029494
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,23.346832,0.008151,0.718932,0.231294,0.158096,0.170818,0.190197,0.094587,1.058302,0.216352


In [26]:
#Kurtosis between consecutive fixations that join with feature vector
kurt_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        kurt_diff = kurtosis(differences)
    else:
        kurt_diff = None

    kurt_diff_data.append({
        'file_key': file_key,
        'f20': kurt_diff
    })

# Create the DataFrame 
kurt_diff_df = pd.DataFrame(kurt_diff_data)

feature_df = feature_df.merge(kurt_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.003726,0.791343,0.130331,0.076023,0.077163,0.188463,0.029011,2.811712,0.056973,6.458946
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.004038,0.579450,0.044324,0.029852,0.028986,0.082984,0.008911,6.079763,0.020567,36.306361
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.035712,0.800112,0.196228,0.136460,0.119380,0.201262,0.044493,1.943941,0.107858,2.472418
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.010676,0.584304,0.093602,0.068154,0.083411,0.077927,0.045900,2.926825,0.092732,15.484313
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.013813,0.784073,0.140735,0.089522,0.080418,0.183063,0.032064,2.735644,0.070823,6.380403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.006312,0.626235,0.047437,0.032359,0.033660,0.087031,0.013811,6.243230,0.026607,38.629103
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.009821,0.767095,0.183198,0.121121,0.109994,0.190756,0.053279,1.878249,0.132005,2.440162
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.003254,0.609310,0.047033,0.032646,0.035231,0.077546,0.015513,6.668649,0.029494,45.506684
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.008151,0.718932,0.231294,0.158096,0.170818,0.190197,0.094587,1.058302,0.216352,-0.065686


In [27]:
#Saccade vector for each file
fpogs_saccade_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    
    result = duckdb.query("""
        SELECT 
            AVG(FPOGX) AS FPOGX, 
            AVG(FPOGY) AS FPOGY, 
            MAX(FPOGS) AS FPOGS, 
            MAX(FPOGD) AS FPOGD, 
            FPOGID
        FROM df
        WHERE FPOGV = '0'
        GROUP BY FPOGID
        ORDER BY FPOGID
    """).to_df()
    
    # Extract vectors from result
    fpogx_vector = result['FPOGX'].tolist()
    fpogy_vector = result['FPOGY'].tolist()
    fpogs_values = result['FPOGS'].tolist()
    fpogd_vector = result['FPOGD'].tolist()
    fpogid_vector = result['FPOGID'].tolist()
    
    # Append structured result
    fpogs_saccade_vector.append({
        'file_key': key,
        'FPOGX': fpogx_vector,
        'FPOGY': fpogy_vector,
        'FPOGS': fpogs_values,
        'FPOGD': fpogd_vector,
        'FPOGID': fpogid_vector
    })

# Convert to DataFrame
fpogs_saccade_vector_df = pd.DataFrame(fpogs_saccade_vector)

# Display the DataFrame
display(fpogs_saccade_vector_df)


,file_key,FPOGX,FPOGY,FPOGS,FPOGD,FPOGID
0,T1_S1_TRY1_VB_SL_LIT,"[0.6081766666666667, 0.46358999999999995, 0.26...","[0.3045466666666667, 0.7783822222222221, 0.939...","[1027.95508, 1028.25623, 1028.62109, 1028.7584...","[0.29431, 0.31482, 0.11023, 0.1748, 0.20728, 0...","[2147, 2148, 2149, 2150, 2151, 2152, 2153, 215..."
1,T1_S1_TRY1_HS_SL_LIT,"[0.6087566666666667, 1.0041711111111111, 1.178...","[0.2997433333333333, 0.37089555555555553, 0.41...","[1099.13293, 1099.75549, 1100.20422, 1100.5535...","[0.61572, 0.40222, 0.34204, 0.18689, 0.18347, ...","[2446, 2447, 2448, 2449, 2450, 2451, 2452, 245..."
2,T1_S1_TRY1_VB_SL_BIG,"[0.5218955555555556, 0.2630266666666667, 0.303...","[0.7372011111111111, 0.9241233333333333, 0.914...","[1117.20703, 1118.19336, 1118.35742, 1118.6656...","[0.93579, 0.1377, 0.28149, 0.22046, 0.26172, 0...","[2499, 2500, 2501, 2502, 2503, 2504, 2505, 250..."
3,T1_S1_TRY1_HS_SL_BIG,"[0.33547, 0.5989766666666667, 1.05433888888888...","[0.31572, 0.20356333333333335, 0.3734611111111...","[1042.94617, 1045.63892, 1046.37512, 1046.5095...","[0.15417, 0.72949, 0.08789, 0.22131, 0.20044, ...","[2202, 2207, 2208, 2209, 2210, 2211, 2212, 221..."
4,T1_S1_TRY1_VB_FA_LIT,"[1.0431050000000002, 0.6273733333333333, 0.521...","[0.5747800000000001, 0.20236666666666667, 0.69...","[1060.85779, 1062.7196, 1064.00647, 1064.37427...","[0.12073, 1.28003, 0.32092, 0.12061, 0.20105, ...","[2295, 2305, 2306, 2307, 2308, 2309, 2310, 231..."
...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[1.0358433333333334, 1.2133066666666668, 1.233...","[0.19562, 0.31321333333333334, 0.2714466666666...","[513.41589, 515.30408, 515.63226, 516.36237, 5...","[1.84009, 0.32172, 0.72406, 0.12732, 0.18842, ...","[1812, 1813, 1814, 1815, 1816, 1817, 1818, 181..."
2372,T34_S1_TRY3_VB_FA_LIT,"[0.5540233333333334, 0.2860533333333333, 0.217...","[0.6069233333333333, 0.7168200000000001, 0.654...","[495.82553, 496.40778, 496.56259, 496.87036, 4...","[0.53595, 0.12921, 0.30081, 0.3483, 0.24155, 0...","[1735, 1736, 1737, 1738, 1739, 1740, 1741, 174..."
2373,T34_S1_TRY3_HS_FA_LIT,"[1.0059822222222223, 1.20838, 1.18519000000000...","[0.23264222222222225, 0.23964, 0.30891, 0.3224...","[626.63928, 628.09259, 628.41333, 628.97729, 6...","[1.40656, 0.3111, 0.55743, 0.27435, 0.15442, 0...","[2225, 2226, 2227, 2228, 2229, 2230, 2231, 223..."
2374,T34_S1_TRY3_VB_FA_BIG,"[0.3418433333333333, 0.34232666666666667, 0.49...","[0.4656255555555556, 0.6259816666666667, 0.610...","[533.45734, 534.32819, 534.63684, 534.8302, 53...","[0.8241, 0.28174, 0.16711, 0.16791, 0.23468, 0...","[1871, 1872, 1873, 1874, 1875, 1876, 1877, 187..."


In [28]:
#Show the saccade data in a nicer way into a csv file

# Flatten the list into a structured tabular format
flattened_fpogs_data = []

for item in fpogs_saccade_vector:
    file_key = item['file_key']
    for fpogid, fpogx, fpogy, fpogs, fpogd in zip(
        item['FPOGID'], item['FPOGX'], item['FPOGY'], item['FPOGS'], item['FPOGD']
    ):
        flattened_fpogs_data.append({
            'file_key': file_key,
            'FPOGID': fpogid,
            'FPOGX': fpogx,
            'FPOGY': fpogy,
            'FPOGS': fpogs,
            'FPOGD': fpogd
        })

# Create the final DataFrame
fpogs_table_df = pd.DataFrame(flattened_fpogs_data)

# Display the table
display(fpogs_table_df)

#Create the csv 
fpogs_table_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\saccade_vector.csv', index=False)


,file_key,FPOGID,FPOGX,FPOGY,FPOGS,FPOGD
0,T1_S1_TRY1_VB_SL_LIT,2147,0.608177,0.304547,1027.95508,0.29431
1,T1_S1_TRY1_VB_SL_LIT,2148,0.463590,0.778382,1028.25623,0.31482
2,T1_S1_TRY1_VB_SL_LIT,2149,0.260463,0.939947,1028.62109,0.11023
3,T1_S1_TRY1_VB_SL_LIT,2150,0.173763,0.901647,1028.75842,0.17480
4,T1_S1_TRY1_VB_SL_LIT,2151,0.235220,0.888807,1028.93994,0.20728
...,...,...,...,...,...,...
138005,T34_S1_TRY3_HS_FA_BIG,2015,1.213767,0.271273,565.80426,0.30811
138006,T34_S1_TRY3_HS_FA_BIG,2016,1.206053,0.261037,566.11884,0.40851
138007,T34_S1_TRY3_HS_FA_BIG,2017,1.228470,0.295370,566.53394,0.14789
138008,T34_S1_TRY3_HS_FA_BIG,2018,1.181017,0.264170,566.68921,0.22009


In [29]:
#Obtain the speed of every saccade in a file

saccade_vector = []

for fix in fpogs_saccade_vector:
    file_key = fix['file_key']
    fpogd_vector = fix['FPOGD']
    fpogs_vector = fix['FPOGS']
    fpogid_vector = fix['FPOGID']
    fpogx_vector = fix['FPOGX']
    fpogy_vector = fix['FPOGY']

    # Build list of saccade speeds
    saccade_speeds = []
    id_pairs = []

    for i in range(len(fpogs_vector) - 1):
        #Euclidean distance between two consecutive saccades
        point1 = [fpogx_vector[i], fpogy_vector[i]]
        point2 = [fpogx_vector[i + 1], fpogy_vector[i + 1]]
        dist = distance.euclidean(point1, point2)
        
        #Time of the saccade. It is obtained from FPOGS[i+1] - FPOGS[i] which include the time of fixation + saccade.
        #To obtain only the saccade time I subract from this difference FPOGD[i] which is the duration of the fixation
        diff = fpogs_vector[i + 1] - fpogs_vector[i]
        time = diff - fpogd_vector[i]

        #Compute the speed
        if time > 0:
            speed = dist / time
        else:
            speed = np.nan  

        saccade_speeds.append(speed)
        id_pairs.append((fpogid_vector[i], fpogid_vector[i + 1]))

    saccade_vector.append({
        'file_key': file_key,
        'FPOGID_pairs': id_pairs,
        'FPOGS_speeds': saccade_speeds
    })

saccade_vector_df = pd.DataFrame(saccade_vector)
display(saccade_vector_df)


,file_key,FPOGID_pairs,FPOGS_speeds
0,T1_S1_TRY1_VB_SL_LIT,"[(2147, 2148), (2148, 2149), (2149, 2150), (21...","[72.42753181127236, 5.18674695737663, 3.497520..."
1,T1_S1_TRY1_HS_SL_LIT,"[(2446, 2447), (2447, 2448), (2448, 2449), (24...","[58.73759245415269, 3.8640221597068387, 8.2272..."
2,T1_S1_TRY1_VB_SL_BIG,"[(2499, 2500), (2500, 2501), (2501, 2502), (25...","[6.317784488580865, 1.5693381478814266, 3.7294..."
3,T1_S1_TRY1_HS_SL_BIG,"[(2202, 2207), (2207, 2208), (2208, 2209), (22...","[0.11281204605962271, 72.43289110337953, 2.468..."
4,T1_S1_TRY1_VB_FA_LIT,"[(2295, 2305), (2305, 2306), (2306, 2307), (23...","[0.3205733264401448, 73.25854873687827, 6.6070..."
...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[(1812, 1813), (1813, 1814), (1814, 1815), (18...","[4.425952107039331, 7.199727605293211, 11.3931..."
2372,T34_S1_TRY3_VB_FA_LIT,"[(1735, 1736), (1736, 1737), (1737, 1738), (17...","[6.255494932282611, 3.6225483381549397, 21.641..."
2373,T34_S1_TRY3_HS_FA_LIT,"[(2225, 2226), (2226, 2227), (2227, 2228), (22...","[4.331951092257692, 7.577663439073629, 2.50964..."
2374,T34_S1_TRY3_VB_FA_BIG,"[(1871, 1872), (1872, 1873), (1873, 1874), (18...","[3.4300928240273687, 5.518831947450474, 6.2331..."


In [30]:
#Minimum saccade speed
min_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        min_speed = np.min(speeds)
    else:
        min_speed = None

    min_speed_data.append({
        'file_key': file_key,
        'f21': min_speed
    })

# Create DataFrame from min speeds
min_speed_df = pd.DataFrame(min_speed_data)

feature_df = feature_df.merge(min_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.791343,0.130331,0.076023,0.077163,0.188463,0.029011,2.811712,0.056973,6.458946,2.660861
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.579450,0.044324,0.029852,0.028986,0.082984,0.008911,6.079763,0.020567,36.306361,1.691588
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.800112,0.196228,0.136460,0.119380,0.201262,0.044493,1.943941,0.107858,2.472418,0.696897
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.584304,0.093602,0.068154,0.083411,0.077927,0.045900,2.926825,0.092732,15.484313,0.112812
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.784073,0.140735,0.089522,0.080418,0.183063,0.032064,2.735644,0.070823,6.380403,0.320573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.626235,0.047437,0.032359,0.033660,0.087031,0.013811,6.243230,0.026607,38.629103,0.547069
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.767095,0.183198,0.121121,0.109994,0.190756,0.053279,1.878249,0.132005,2.440162,0.946394
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.609310,0.047033,0.032646,0.035231,0.077546,0.015513,6.668649,0.029494,45.506684,2.258086
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.718932,0.231294,0.158096,0.170818,0.190197,0.094587,1.058302,0.216352,-0.065686,0.362771


In [31]:
#Maximim saccade speed
max_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        max_speed = np.max(speeds)
    else:
        max_speed = None

    max_speed_data.append({
        'file_key': file_key,
        'f22': max_speed
    })

# Create DataFrame from max speeds
max_speed_df = pd.DataFrame(max_speed_data)

feature_df = feature_df.merge(max_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.130331,0.076023,0.077163,0.188463,0.029011,2.811712,0.056973,6.458946,2.660861,94.253395
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.044324,0.029852,0.028986,0.082984,0.008911,6.079763,0.020567,36.306361,1.691588,144.562838
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.196228,0.136460,0.119380,0.201262,0.044493,1.943941,0.107858,2.472418,0.696897,74.235271
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.093602,0.068154,0.083411,0.077927,0.045900,2.926825,0.092732,15.484313,0.112812,108.195641
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.140735,0.089522,0.080418,0.183063,0.032064,2.735644,0.070823,6.380403,0.320573,88.652115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.047437,0.032359,0.033660,0.087031,0.013811,6.243230,0.026607,38.629103,0.547069,118.206935
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.183198,0.121121,0.109994,0.190756,0.053279,1.878249,0.132005,2.440162,0.946394,95.087541
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.047033,0.032646,0.035231,0.077546,0.015513,6.668649,0.029494,45.506684,2.258086,12.578635
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.231294,0.158096,0.170818,0.190197,0.094587,1.058302,0.216352,-0.065686,0.362771,93.195696


In [32]:
#Arithmetic mean saccade speed
mean_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        mean_speed = np.mean(speeds)
    else:
        mean_speed = None

    mean_speed_data.append({
        'file_key': file_key,
        'f23': mean_speed
    })

# Create DataFrame from mean speeds
mean_speed_df = pd.DataFrame(mean_speed_data)

feature_df = feature_df.merge(mean_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.076023,0.077163,0.188463,0.029011,2.811712,0.056973,6.458946,2.660861,94.253395,16.230771
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.029852,0.028986,0.082984,0.008911,6.079763,0.020567,36.306361,1.691588,144.562838,17.409787
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.136460,0.119380,0.201262,0.044493,1.943941,0.107858,2.472418,0.696897,74.235271,17.365937
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.068154,0.083411,0.077927,0.045900,2.926825,0.092732,15.484313,0.112812,108.195641,12.826917
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.089522,0.080418,0.183063,0.032064,2.735644,0.070823,6.380403,0.320573,88.652115,16.393740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.032359,0.033660,0.087031,0.013811,6.243230,0.026607,38.629103,0.547069,118.206935,10.808403
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.121121,0.109994,0.190756,0.053279,1.878249,0.132005,2.440162,0.946394,95.087541,18.755266
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.032646,0.035231,0.077546,0.015513,6.668649,0.029494,45.506684,2.258086,12.578635,7.202338
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.158096,0.170818,0.190197,0.094587,1.058302,0.216352,-0.065686,0.362771,93.195696,19.822593


In [33]:
#Geometric mean saccade speed
gmean_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        gmean_speed = gmean(speeds)
    else:
        gmean_speed = None

    gmean_speed_data.append({
        'file_key': file_key,
        'f24': gmean_speed
    })

# Create DataFrame from gmean speeds
gmean_speed_df = pd.DataFrame(gmean_speed_data)

feature_df = feature_df.merge(gmean_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.077163,0.188463,0.029011,2.811712,0.056973,6.458946,2.660861,94.253395,16.230771,10.506072
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.028986,0.082984,0.008911,6.079763,0.020567,36.306361,1.691588,144.562838,17.409787,8.561762
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.119380,0.201262,0.044493,1.943941,0.107858,2.472418,0.696897,74.235271,17.365937,10.750664
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.083411,0.077927,0.045900,2.926825,0.092732,15.484313,0.112812,108.195641,12.826917,7.485284
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.080418,0.183063,0.032064,2.735644,0.070823,6.380403,0.320573,88.652115,16.393740,10.697536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.033660,0.087031,0.013811,6.243230,0.026607,38.629103,0.547069,118.206935,10.808403,6.118575
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.109994,0.190756,0.053279,1.878249,0.132005,2.440162,0.946394,95.087541,18.755266,11.280071
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.035231,0.077546,0.015513,6.668649,0.029494,45.506684,2.258086,12.578635,7.202338,6.647696
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.170818,0.190197,0.094587,1.058302,0.216352,-0.065686,0.362771,93.195696,19.822593,10.285791


In [34]:
#Median saccade speed
median_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        median_speed = np.median(speeds)
    else:
        median_speed = None

    median_speed_data.append({
        'file_key': file_key,
        'f25': median_speed
    })

# Create DataFrame from median speeds
median_speed_df = pd.DataFrame(median_speed_data)

feature_df = feature_df.merge(median_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.188463,0.029011,2.811712,0.056973,6.458946,2.660861,94.253395,16.230771,10.506072,9.368855
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.082984,0.008911,6.079763,0.020567,36.306361,1.691588,144.562838,17.409787,8.561762,7.678763
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.201262,0.044493,1.943941,0.107858,2.472418,0.696897,74.235271,17.365937,10.750664,13.016494
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.077927,0.045900,2.926825,0.092732,15.484313,0.112812,108.195641,12.826917,7.485284,8.412300
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.183063,0.032064,2.735644,0.070823,6.380403,0.320573,88.652115,16.393740,10.697536,10.497244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.087031,0.013811,6.243230,0.026607,38.629103,0.547069,118.206935,10.808403,6.118575,6.200144
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.190756,0.053279,1.878249,0.132005,2.440162,0.946394,95.087541,18.755266,11.280071,11.322407
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.077546,0.015513,6.668649,0.029494,45.506684,2.258086,12.578635,7.202338,6.647696,7.178951
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.190197,0.094587,1.058302,0.216352,-0.065686,0.362771,93.195696,19.822593,10.285791,9.302406


In [35]:
#STD saccade speed
std_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        std_speed = np.std(speeds)
    else:
        std_speed = None

    std_speed_data.append({
        'file_key': file_key,
        'f26': std_speed
    })

# Create DataFrame from std speeds
std_speed_df = pd.DataFrame(std_speed_data)

feature_df = feature_df.merge(std_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.029011,2.811712,0.056973,6.458946,2.660861,94.253395,16.230771,10.506072,9.368855,22.143950
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.008911,6.079763,0.020567,36.306361,1.691588,144.562838,17.409787,8.561762,7.678763,33.659091
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.044493,1.943941,0.107858,2.472418,0.696897,74.235271,17.365937,10.750664,13.016494,17.803497
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.045900,2.926825,0.092732,15.484313,0.112812,108.195641,12.826917,7.485284,8.412300,14.822180
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.032064,2.735644,0.070823,6.380403,0.320573,88.652115,16.393740,10.697536,10.497244,19.899162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.013811,6.243230,0.026607,38.629103,0.547069,118.206935,10.808403,6.118575,6.200144,22.487589
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.053279,1.878249,0.132005,2.440162,0.946394,95.087541,18.755266,11.280071,11.322407,21.954502
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.015513,6.668649,0.029494,45.506684,2.258086,12.578635,7.202338,6.647696,7.178951,2.612026
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.094587,1.058302,0.216352,-0.065686,0.362771,93.195696,19.822593,10.285791,9.302406,21.638664


In [36]:
#MAD saccade speed
mad_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        mad_speed = median_abs_deviation(speeds, scale=1)
    else:
        mad_speed = None

    mad_speed_data.append({
        'file_key': file_key,
        'f27': mad_speed
    })

# Create DataFrame from mad speeds
mad_speed_df = pd.DataFrame(mad_speed_data)

feature_df = feature_df.merge(mad_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,2.811712,0.056973,6.458946,2.660861,94.253395,16.230771,10.506072,9.368855,22.143950,2.833518
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,6.079763,0.020567,36.306361,1.691588,144.562838,17.409787,8.561762,7.678763,33.659091,2.362886
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.943941,0.107858,2.472418,0.696897,74.235271,17.365937,10.750664,13.016494,17.803497,8.808332
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,2.926825,0.092732,15.484313,0.112812,108.195641,12.826917,7.485284,8.412300,14.822180,6.580242
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,2.735644,0.070823,6.380403,0.320573,88.652115,16.393740,10.697536,10.497244,19.899162,4.091115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,6.243230,0.026607,38.629103,0.547069,118.206935,10.808403,6.118575,6.200144,22.487589,1.755536
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.878249,0.132005,2.440162,0.946394,95.087541,18.755266,11.280071,11.322407,21.954502,6.195296
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,6.668649,0.029494,45.506684,2.258086,12.578635,7.202338,6.647696,7.178951,2.612026,2.057708
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.058302,0.216352,-0.065686,0.362771,93.195696,19.822593,10.285791,9.302406,21.638664,7.115941


In [37]:
#Skewness saccade speed
skew_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        skew_speed = skew(speeds)
    else:
        skew_speed = None

    skew_speed_data.append({
        'file_key': file_key,
        'f28': skew_speed
    })

# Create DataFrame from skew speeds
skew_speed_df = pd.DataFrame(skew_speed_data)

feature_df = feature_df.merge(skew_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.056973,6.458946,2.660861,94.253395,16.230771,10.506072,9.368855,22.143950,2.833518,2.732383
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.020567,36.306361,1.691588,144.562838,17.409787,8.561762,7.678763,33.659091,2.362886,3.175859
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.107858,2.472418,0.696897,74.235271,17.365937,10.750664,13.016494,17.803497,8.808332,1.866832
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.092732,15.484313,0.112812,108.195641,12.826917,7.485284,8.412300,14.822180,6.580242,3.731514
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.070823,6.380403,0.320573,88.652115,16.393740,10.697536,10.497244,19.899162,4.091115,2.742697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.026607,38.629103,0.547069,118.206935,10.808403,6.118575,6.200144,22.487589,1.755536,4.382837
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.132005,2.440162,0.946394,95.087541,18.755266,11.280071,11.322407,21.954502,6.195296,2.162583
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.029494,45.506684,2.258086,12.578635,7.202338,6.647696,7.178951,2.612026,2.057708,-0.117216
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.216352,-0.065686,0.362771,93.195696,19.822593,10.285791,9.302406,21.638664,7.115941,1.458420


In [38]:
#IQR saccade speed
iqr_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        iqr_speed = iqr(speeds)
    else:
        iqr_speed = None

    iqr_speed_data.append({
        'file_key': file_key,
        'f29': iqr_speed
    })

# Create DataFrame from iqr speeds
iqr_speed_df = pd.DataFrame(iqr_speed_data)

feature_df = feature_df.merge(iqr_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,6.458946,2.660861,94.253395,16.230771,10.506072,9.368855,22.143950,2.833518,2.732383,5.535011
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,36.306361,1.691588,144.562838,17.409787,8.561762,7.678763,33.659091,2.362886,3.175859,4.205481
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,2.472418,0.696897,74.235271,17.365937,10.750664,13.016494,17.803497,8.808332,1.866832,17.689187
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,15.484313,0.112812,108.195641,12.826917,7.485284,8.412300,14.822180,6.580242,3.731514,15.095052
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,6.380403,0.320573,88.652115,16.393740,10.697536,10.497244,19.899162,4.091115,2.742697,8.361635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,38.629103,0.547069,118.206935,10.808403,6.118575,6.200144,22.487589,1.755536,4.382837,3.543484
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,2.440162,0.946394,95.087541,18.755266,11.280071,11.322407,21.954502,6.195296,2.162583,12.938040
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,45.506684,2.258086,12.578635,7.202338,6.647696,7.178951,2.612026,2.057708,-0.117216,3.911825
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,-0.065686,0.362771,93.195696,19.822593,10.285791,9.302406,21.638664,7.115941,1.458420,24.898276


In [39]:
#Kurtosis saccade speed
kurt_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        kurt_speed = kurtosis(speeds)
    else:
        kurt_speed = None

    kurt_speed_data.append({
        'file_key': file_key,
        'f30': kurt_speed
    })

# Create DataFrame from kurt speeds
kurt_speed_df = pd.DataFrame(kurt_speed_data)

feature_df = feature_df.merge(kurt_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,2.660861,94.253395,16.230771,10.506072,9.368855,22.143950,2.833518,2.732383,5.535011,5.843646
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.691588,144.562838,17.409787,8.561762,7.678763,33.659091,2.362886,3.175859,4.205481,8.565004
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.696897,74.235271,17.365937,10.750664,13.016494,17.803497,8.808332,1.866832,17.689187,2.996958
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.112812,108.195641,12.826917,7.485284,8.412300,14.822180,6.580242,3.731514,15.095052,19.442884
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.320573,88.652115,16.393740,10.697536,10.497244,19.899162,4.091115,2.742697,8.361635,6.527002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.547069,118.206935,10.808403,6.118575,6.200144,22.487589,1.755536,4.382837,3.543484,17.535290
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.946394,95.087541,18.755266,11.280071,11.322407,21.954502,6.195296,2.162583,12.938040,3.898355
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,2.258086,12.578635,7.202338,6.647696,7.178951,2.612026,2.057708,-0.117216,3.911825,-0.900642
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.362771,93.195696,19.822593,10.285791,9.302406,21.638664,7.115941,1.458420,24.898276,1.432311


In [40]:
#Scanpath lenght for every file
#The scanpath lenght is computed summing the euclidean distance of every consecutive point in the file

# Load fixation csv
csv_path = r"C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\fixation_vector.csv"
df = pd.read_csv(csv_path)

fpogs_scan_vector = []

# Group by 'file_key'
for file_key, group in df.groupby('file_key'):
    # Sort by fixation ID
    group = group.sort_values('FPOGID')
    
    fpogx_vector = group['FPOGX'].tolist()
    fpogy_vector = group['FPOGY'].tolist()

    tot = 0
    for i in range(len(fpogx_vector) - 1):
        point1 = [fpogx_vector[i], fpogy_vector[i]]
        point2 = [fpogx_vector[i + 1], fpogy_vector[i + 1]]
        dist = distance.euclidean(point1, point2)
        tot += dist

    fpogs_scan_vector.append({
        'file_key': file_key,
        'f31': tot
    })

fpogs_scan_vector_df = pd.DataFrame(fpogs_scan_vector)
feature_df = feature_df.merge(fpogs_scan_vector_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,94.253395,16.230771,10.506072,9.368855,22.143950,2.833518,2.732383,5.535011,5.843646,7.298560
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,144.562838,17.409787,8.561762,7.678763,33.659091,2.362886,3.175859,4.205481,8.565004,1.950242
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,74.235271,17.365937,10.750664,13.016494,17.803497,8.808332,1.866832,17.689187,2.996958,11.381240
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,108.195641,12.826917,7.485284,8.412300,14.822180,6.580242,3.731514,15.095052,19.442884,8.236999
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,88.652115,16.393740,10.697536,10.497244,19.899162,4.091115,2.742697,8.361635,6.527002,8.866278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,118.206935,10.808403,6.118575,6.200144,22.487589,1.755536,4.382837,3.543484,17.535290,2.229547
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,95.087541,18.755266,11.280071,11.322407,21.954502,6.195296,2.162583,12.938040,3.898355,12.823836
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,12.578635,7.202338,6.647696,7.178951,2.612026,2.057708,-0.117216,3.911825,-0.900642,2.680895
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,93.195696,19.822593,10.285791,9.302406,21.638664,7.115941,1.458420,24.898276,1.432311,17.809671


In [41]:
#List of all valid(LPV = 1) left pupil diameter(LPD) for each file
lpd_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT LPD 
        FROM df
        WHERE LPV = '1'
    """).to_df()

    # Extract vectors
    lpd_list = result['LPD'].tolist()

    # Append result 
    lpd_vector.append({
        'file_key': key,
        'LPD': lpd_list
    })

# Create the DataFrame
lpd_vector_df = pd.DataFrame(lpd_vector)

# Display the DataFrame
display(lpd_vector_df)


,file_key,LPD
0,T1_S1_TRY1_VB_SL_LIT,"[18.33092, 18.23724, 20.23254, 20.18498, 20.33..."
1,T1_S1_TRY1_HS_SL_LIT,"[20.73535, 20.51113, 20.60152, 20.485, 20.4977..."
2,T1_S1_TRY1_VB_SL_BIG,"[17.84583, 18.04667, 18.0429, 17.9603, 17.9667..."
3,T1_S1_TRY1_HS_SL_BIG,"[16.78424, 18.47666, 18.58866, 18.46712, 18.70..."
4,T1_S1_TRY1_VB_FA_LIT,"[20.1371, 20.03338, 19.02217, 19.03521, 19.026..."
...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[25.5071, 25.84493, 25.87336, 26.05766, 26.016..."
2372,T34_S1_TRY3_VB_FA_LIT,"[27.04837, 26.95319, 26.14511, 26.04167, 26.04..."
2373,T34_S1_TRY3_HS_FA_LIT,"[22.13408, 22.56157, 22.2504, 22.18439, 22.286..."
2374,T34_S1_TRY3_VB_FA_BIG,"[25.01905, 25.1654, 25.01192, 24.98197, 25.010..."


In [42]:
#Minimum left pupil diameter
min_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        min_lpd = np.min(lpd)
    else:
        min_lpd = None

    min_lpd_data.append({
        'file_key': file_key,
        'f32': min_lpd
    })

# Create DataFrame from min differences
min_lpd_df = pd.DataFrame(min_lpd_data)

feature_df = feature_df.merge(min_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,16.230771,10.506072,9.368855,22.143950,2.833518,2.732383,5.535011,5.843646,7.298560,14.97845
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,17.409787,8.561762,7.678763,33.659091,2.362886,3.175859,4.205481,8.565004,1.950242,9.10561
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,17.365937,10.750664,13.016494,17.803497,8.808332,1.866832,17.689187,2.996958,11.381240,14.20694
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,12.826917,7.485284,8.412300,14.822180,6.580242,3.731514,15.095052,19.442884,8.236999,16.78424
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,16.393740,10.697536,10.497244,19.899162,4.091115,2.742697,8.361635,6.527002,8.866278,16.44751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,10.808403,6.118575,6.200144,22.487589,1.755536,4.382837,3.543484,17.535290,2.229547,7.14804
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,18.755266,11.280071,11.322407,21.954502,6.195296,2.162583,12.938040,3.898355,12.823836,18.45790
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,7.202338,6.647696,7.178951,2.612026,2.057708,-0.117216,3.911825,-0.900642,2.680895,20.82297
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,19.822593,10.285791,9.302406,21.638664,7.115941,1.458420,24.898276,1.432311,17.809671,21.63134


In [43]:
#Maximum left pupil diameter
max_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        max_lpd = np.max(lpd)
    else:
        max_lpd = None

    max_lpd_data.append({
        'file_key': file_key,
        'f33': max_lpd
    })

# Create DataFrame 
max_lpd_df = pd.DataFrame(max_lpd_data)

feature_df = feature_df.merge(max_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,10.506072,9.368855,22.143950,2.833518,2.732383,5.535011,5.843646,7.298560,14.97845,21.82685
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,8.561762,7.678763,33.659091,2.362886,3.175859,4.205481,8.565004,1.950242,9.10561,23.06998
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,10.750664,13.016494,17.803497,8.808332,1.866832,17.689187,2.996958,11.381240,14.20694,22.62979
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,7.485284,8.412300,14.822180,6.580242,3.731514,15.095052,19.442884,8.236999,16.78424,22.31914
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,10.697536,10.497244,19.899162,4.091115,2.742697,8.361635,6.527002,8.866278,16.44751,21.14130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,6.118575,6.200144,22.487589,1.755536,4.382837,3.543484,17.535290,2.229547,7.14804,26.76938
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,11.280071,11.322407,21.954502,6.195296,2.162583,12.938040,3.898355,12.823836,18.45790,27.74957
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,6.647696,7.178951,2.612026,2.057708,-0.117216,3.911825,-0.900642,2.680895,20.82297,24.67662
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,10.285791,9.302406,21.638664,7.115941,1.458420,24.898276,1.432311,17.809671,21.63134,26.75565


In [44]:
#Arithmetic mean of left pupil diameter
mean_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        mean_lpd = np.mean(lpd)
    else:
        mean_lpd = None

    mean_lpd_data.append({
        'file_key': file_key,
        'f34': mean_lpd
    })

# Create DataFrame 
mean_lpd_df = pd.DataFrame(mean_lpd_data)

feature_df = feature_df.merge(mean_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,9.368855,22.143950,2.833518,2.732383,5.535011,5.843646,7.298560,14.97845,21.82685,18.883407
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,7.678763,33.659091,2.362886,3.175859,4.205481,8.565004,1.950242,9.10561,23.06998,21.053556
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,13.016494,17.803497,8.808332,1.866832,17.689187,2.996958,11.381240,14.20694,22.62979,19.978792
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,8.412300,14.822180,6.580242,3.731514,15.095052,19.442884,8.236999,16.78424,22.31914,20.575229
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,10.497244,19.899162,4.091115,2.742697,8.361635,6.527002,8.866278,16.44751,21.14130,18.973598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,6.200144,22.487589,1.755536,4.382837,3.543484,17.535290,2.229547,7.14804,26.76938,24.487939
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,11.322407,21.954502,6.195296,2.162583,12.938040,3.898355,12.823836,18.45790,27.74957,23.589736
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,7.178951,2.612026,2.057708,-0.117216,3.911825,-0.900642,2.680895,20.82297,24.67662,23.541069
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,9.302406,21.638664,7.115941,1.458420,24.898276,1.432311,17.809671,21.63134,26.75565,24.038749


In [45]:
#Geometric mean of left pupil diameter
gmean_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        gmean_lpd = gmean(lpd)
    else:
        gmean_lpd = None

    gmean_lpd_data.append({
        'file_key': file_key,
        'f35': gmean_lpd
    })

# Create DataFrame 
gmean_lpd_df = pd.DataFrame(gmean_lpd_data)

feature_df = feature_df.merge(gmean_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,22.143950,2.833518,2.732383,5.535011,5.843646,7.298560,14.97845,21.82685,18.883407,18.793943
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,33.659091,2.362886,3.175859,4.205481,8.565004,1.950242,9.10561,23.06998,21.053556,21.035456
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,17.803497,8.808332,1.866832,17.689187,2.996958,11.381240,14.20694,22.62979,19.978792,19.927327
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,14.822180,6.580242,3.731514,15.095052,19.442884,8.236999,16.78424,22.31914,20.575229,20.561464
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,19.899162,4.091115,2.742697,8.361635,6.527002,8.866278,16.44751,21.14130,18.973598,18.944220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,22.487589,1.755536,4.382837,3.543484,17.535290,2.229547,7.14804,26.76938,24.487939,24.468674
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,21.954502,6.195296,2.162583,12.938040,3.898355,12.823836,18.45790,27.74957,23.589736,23.500054
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,2.612026,2.057708,-0.117216,3.911825,-0.900642,2.680895,20.82297,24.67662,23.541069,23.530741
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,21.638664,7.115941,1.458420,24.898276,1.432311,17.809671,21.63134,26.75565,24.038749,24.007471


In [46]:
#Median of left pupil diameter
median_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        median_lpd = np.median(lpd)
    else:
        median_lpd = None

    median_lpd_data.append({
        'file_key': file_key,
        'f36': median_lpd
    })

# Create DataFrame 
median_lpd_df = pd.DataFrame(median_lpd_data)

feature_df = feature_df.merge(median_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,2.833518,2.732383,5.535011,5.843646,7.298560,14.97845,21.82685,18.883407,18.793943,18.824400
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,2.362886,3.175859,4.205481,8.565004,1.950242,9.10561,23.06998,21.053556,21.035456,21.097650
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,8.808332,1.866832,17.689187,2.996958,11.381240,14.20694,22.62979,19.978792,19.927327,20.053690
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,6.580242,3.731514,15.095052,19.442884,8.236999,16.78424,22.31914,20.575229,20.561464,20.493495
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,4.091115,2.742697,8.361635,6.527002,8.866278,16.44751,21.14130,18.973598,18.944220,19.005680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,1.755536,4.382837,3.543484,17.535290,2.229547,7.14804,26.76938,24.487939,24.468674,24.430845
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,6.195296,2.162583,12.938040,3.898355,12.823836,18.45790,27.74957,23.589736,23.500054,23.465935
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,2.057708,-0.117216,3.911825,-0.900642,2.680895,20.82297,24.67662,23.541069,23.530741,23.658890
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,7.115941,1.458420,24.898276,1.432311,17.809671,21.63134,26.75565,24.038749,24.007471,23.797570


In [47]:
#STD of left pupil diameter
std_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        std_lpd = np.std(lpd)
    else:
        median_lpd = None

    std_lpd_data.append({
        'file_key': file_key,
        'f37': std_lpd
    })

# Create DataFrame 
std_lpd_df = pd.DataFrame(std_lpd_data)

feature_df = feature_df.merge(std_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,2.732383,5.535011,5.843646,7.298560,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,3.175859,4.205481,8.565004,1.950242,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.866832,17.689187,2.996958,11.381240,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,3.731514,15.095052,19.442884,8.236999,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,2.742697,8.361635,6.527002,8.866278,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,4.382837,3.543484,17.535290,2.229547,7.14804,26.76938,24.487939,24.468674,24.430845,0.855238
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,2.162583,12.938040,3.898355,12.823836,18.45790,27.74957,23.589736,23.500054,23.465935,2.062644
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,-0.117216,3.911825,-0.900642,2.680895,20.82297,24.67662,23.541069,23.530741,23.658890,0.686083
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.458420,24.898276,1.432311,17.809671,21.63134,26.75565,24.038749,24.007471,23.797570,1.233973


In [48]:
#MAD of left pupil diameter
mad_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        mad_lpd = median_abs_deviation(lpd, scale=1)
    else:
        median_lpd = None

    mad_lpd_data.append({
        'file_key': file_key,
        'f38': mad_lpd
    })

# Create DataFrame 
mad_lpd_df = pd.DataFrame(mad_lpd_data)

feature_df = feature_df.merge(mad_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,5.535011,5.843646,7.298560,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,4.205481,8.565004,1.950242,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,17.689187,2.996958,11.381240,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,15.095052,19.442884,8.236999,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,8.361635,6.527002,8.866278,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,3.543484,17.535290,2.229547,7.14804,26.76938,24.487939,24.468674,24.430845,0.855238,0.319515
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,12.938040,3.898355,12.823836,18.45790,27.74957,23.589736,23.500054,23.465935,2.062644,1.899110
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,3.911825,-0.900642,2.680895,20.82297,24.67662,23.541069,23.530741,23.658890,0.686083,0.338015
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,24.898276,1.432311,17.809671,21.63134,26.75565,24.038749,24.007471,23.797570,1.233973,0.887340


In [49]:
#SKEWNESS of left pupil diameter
skew_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        skew_lpd = skew(lpd)
    else:
        median_lpd = None

    skew_lpd_data.append({
        'file_key': file_key,
        'f39': skew_lpd
    })

# Create DataFrame 
skew_lpd_df = pd.DataFrame(skew_lpd_data)

feature_df = feature_df.merge(skew_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,5.843646,7.298560,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,8.565004,1.950242,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,2.996958,11.381240,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,19.442884,8.236999,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,6.527002,8.866278,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,17.535290,2.229547,7.14804,26.76938,24.487939,24.468674,24.430845,0.855238,0.319515,-6.132153
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,3.898355,12.823836,18.45790,27.74957,23.589736,23.500054,23.465935,2.062644,1.899110,0.167952
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,-0.900642,2.680895,20.82297,24.67662,23.541069,23.530741,23.658890,0.686083,0.338015,-1.574758
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.432311,17.809671,21.63134,26.75565,24.038749,24.007471,23.797570,1.233973,0.887340,0.424201


In [50]:
#IQR of left pupil diameter
iqr_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        iqr_lpd = iqr(lpd)
    else:
        median_lpd = None

    iqr_lpd_data.append({
        'file_key': file_key,
        'f40': iqr_lpd
    })

# Create DataFrame 
iqr_lpd_df = pd.DataFrame(iqr_lpd_data)

feature_df = feature_df.merge(iqr_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,7.298560,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.950242,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,11.381240,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,8.236999,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,8.866278,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,2.229547,7.14804,26.76938,24.487939,24.468674,24.430845,0.855238,0.319515,-6.132153,0.638328
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,12.823836,18.45790,27.74957,23.589736,23.500054,23.465935,2.062644,1.899110,0.167952,3.845505
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,2.680895,20.82297,24.67662,23.541069,23.530741,23.658890,0.686083,0.338015,-1.574758,0.679110
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,17.809671,21.63134,26.75565,24.038749,24.007471,23.797570,1.233973,0.887340,0.424201,1.950445


In [51]:
#Kurtosis of left pupil diameter
kurt_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        kurt_lpd = kurtosis(lpd)
    else:
        median_lpd = None

    kurt_lpd_data.append({
        'file_key': file_key,
        'f41': kurt_lpd
    })

# Create DataFrame 
kurt_lpd_df = pd.DataFrame(kurt_lpd_data)

feature_df = feature_df.merge(kurt_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,7.14804,26.76938,24.487939,24.468674,24.430845,0.855238,0.319515,-6.132153,0.638328,108.547275
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,18.45790,27.74957,23.589736,23.500054,23.465935,2.062644,1.899110,0.167952,3.845505,-1.347748
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,20.82297,24.67662,23.541069,23.530741,23.658890,0.686083,0.338015,-1.574758,0.679110,2.944222
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,21.63134,26.75565,24.038749,24.007471,23.797570,1.233973,0.887340,0.424201,1.950445,-0.852725


In [52]:
#List of all valid(RPV = 1) right pupil diameter(RPD) for each file
rpd_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT RPD 
        FROM df
        WHERE RPV = '1'
    """).to_df()

    # Extract vectors
    rpd_list = result['RPD'].tolist()

    # Append result 
    rpd_vector.append({
        'file_key': key,
        'RPD': rpd_list
    })

# Create the DataFrame
rpd_vector_df = pd.DataFrame(rpd_vector)

# Display the DataFrame
display(rpd_vector_df)


,file_key,RPD
0,T1_S1_TRY1_VB_SL_LIT,"[17.2877, 17.82784, 18.85434, 18.85434, 18.854..."
1,T1_S1_TRY1_HS_SL_LIT,"[19.8148, 19.63718, 19.91414, 19.88341, 20.081..."
2,T1_S1_TRY1_VB_SL_BIG,"[16.83182, 17.0567, 16.94397, 16.91181, 16.819..."
3,T1_S1_TRY1_HS_SL_BIG,"[16.64259, 18.41689, 18.45479, 18.44613, 18.48..."
4,T1_S1_TRY1_VB_FA_LIT,"[19.08659, 18.89192, 18.82117, 18.85699, 18.70..."
...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[22.87852, 22.68684, 22.83361, 22.72127, 22.62..."
2372,T34_S1_TRY3_VB_FA_LIT,"[23.37141, 23.16466, 22.33815, 22.55819, 22.25..."
2373,T34_S1_TRY3_HS_FA_LIT,"[21.23022, 21.22306, 20.98546, 20.94403, 21.03..."
2374,T34_S1_TRY3_VB_FA_BIG,"[21.60349, 21.41558, 21.49331, 21.49331, 21.84..."


In [53]:
#Minimum right pupil diameter
min_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        min_rpd = np.min(rpd)
    else:
        min_rpd = None

    min_rpd_data.append({
        'file_key': file_key,
        'f42': min_rpd
    })

# Create DataFrame from min differences
min_rpd_df = pd.DataFrame(min_rpd_data)

feature_df = feature_df.merge(min_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,26.76938,24.487939,24.468674,24.430845,0.855238,0.319515,-6.132153,0.638328,108.547275,7.03303
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,27.74957,23.589736,23.500054,23.465935,2.062644,1.899110,0.167952,3.845505,-1.347748,11.05365
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,24.67662,23.541069,23.530741,23.658890,0.686083,0.338015,-1.574758,0.679110,2.944222,17.58587
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,26.75565,24.038749,24.007471,23.797570,1.233973,0.887340,0.424201,1.950445,-0.852725,19.73306


In [54]:
#Maximum right pupil diameter
max_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        max_rpd = np.max(rpd)
    else:
        max_rpd = None

    max_rpd_data.append({
        'file_key': file_key,
        'f43': max_rpd
    })

# Create DataFrame 
max_rpd_df = pd.DataFrame(max_rpd_data)

feature_df = feature_df.merge(max_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,24.487939,24.468674,24.430845,0.855238,0.319515,-6.132153,0.638328,108.547275,7.03303,24.56396
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,23.589736,23.500054,23.465935,2.062644,1.899110,0.167952,3.845505,-1.347748,11.05365,25.73604
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,23.541069,23.530741,23.658890,0.686083,0.338015,-1.574758,0.679110,2.944222,17.58587,23.34963
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,24.038749,24.007471,23.797570,1.233973,0.887340,0.424201,1.950445,-0.852725,19.73306,24.60662


In [55]:
#Arithmetic mean of right pupil diameter
mean_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        mean_rpd = np.mean(rpd)
    else:
        mean_rpd = None

    mean_rpd_data.append({
        'file_key': file_key,
        'f44': mean_rpd
    })

# Create DataFrame 
mean_rpd_df = pd.DataFrame(mean_rpd_data)

feature_df = feature_df.merge(mean_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,24.468674,24.430845,0.855238,0.319515,-6.132153,0.638328,108.547275,7.03303,24.56396,22.308431
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,23.500054,23.465935,2.062644,1.899110,0.167952,3.845505,-1.347748,11.05365,25.73604,21.549576
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,23.530741,23.658890,0.686083,0.338015,-1.574758,0.679110,2.944222,17.58587,23.34963,21.520051
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,24.007471,23.797570,1.233973,0.887340,0.424201,1.950445,-0.852725,19.73306,24.60662,22.026777


In [56]:
#Geometric mean of right pupil diameter
gmean_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        gmean_rpd = gmean(rpd)
    else:
        gmean_rpd = None

    gmean_rpd_data.append({
        'file_key': file_key,
        'f45': gmean_rpd
    })

# Create DataFrame 
gmean_rpd_df = pd.DataFrame(gmean_rpd_data)

feature_df = feature_df.merge(gmean_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,24.430845,0.855238,0.319515,-6.132153,0.638328,108.547275,7.03303,24.56396,22.308431,22.293014
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,23.465935,2.062644,1.899110,0.167952,3.845505,-1.347748,11.05365,25.73604,21.549576,21.485345
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,23.658890,0.686083,0.338015,-1.574758,0.679110,2.944222,17.58587,23.34963,21.520051,21.483555
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,23.797570,1.233973,0.887340,0.424201,1.950445,-0.852725,19.73306,24.60662,22.026777,22.012955


In [57]:
#Median of right pupil diameter
median_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        median_rpd = np.median(rpd)
    else:
        median_rpd = None

    median_rpd_data.append({
        'file_key': file_key,
        'f46': median_rpd
    })

# Create DataFrame 
median_rpd_df = pd.DataFrame(median_rpd_data)

feature_df = feature_df.merge(median_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.855238,0.319515,-6.132153,0.638328,108.547275,7.03303,24.56396,22.308431,22.293014,22.294505
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,2.062644,1.899110,0.167952,3.845505,-1.347748,11.05365,25.73604,21.549576,21.485345,21.603905
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.686083,0.338015,-1.574758,0.679110,2.944222,17.58587,23.34963,21.520051,21.483555,21.861390
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.233973,0.887340,0.424201,1.950445,-0.852725,19.73306,24.60662,22.026777,22.012955,21.909645


In [58]:
#STD of right pupil diameter
std_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        std_rpd = np.std(rpd)
    else:
        median_rpd = None

    std_rpd_data.append({
        'file_key': file_key,
        'f47': std_rpd
    })

# Create DataFrame 
std_rpd_df = pd.DataFrame(std_rpd_data)

feature_df = feature_df.merge(std_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.319515,-6.132153,0.638328,108.547275,7.03303,24.56396,22.308431,22.293014,22.294505,0.720713
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.899110,0.167952,3.845505,-1.347748,11.05365,25.73604,21.549576,21.485345,21.603905,1.663640
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.338015,-1.574758,0.679110,2.944222,17.58587,23.34963,21.520051,21.483555,21.861390,1.212035
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.887340,0.424201,1.950445,-0.852725,19.73306,24.60662,22.026777,22.012955,21.909645,0.787555


In [59]:
#MAD of right pupil diameter
mad_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        mad_rpd = median_abs_deviation(rpd, scale=1)
    else:
        median_rpd = None

    mad_rpd_data.append({
        'file_key': file_key,
        'f48': mad_rpd
    })

# Create DataFrame 
mad_rpd_df = pd.DataFrame(mad_rpd_data)

feature_df = feature_df.merge(mad_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,-6.132153,0.638328,108.547275,7.03303,24.56396,22.308431,22.293014,22.294505,0.720713,0.344680
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.167952,3.845505,-1.347748,11.05365,25.73604,21.549576,21.485345,21.603905,1.663640,1.242880
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,-1.574758,0.679110,2.944222,17.58587,23.34963,21.520051,21.483555,21.861390,1.212035,0.500950
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.424201,1.950445,-0.852725,19.73306,24.60662,22.026777,22.012955,21.909645,0.787555,0.371335


In [60]:
#SKEWNESS of right pupil diameter
skew_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        skew_rpd = skew(rpd)
    else:
        median_rpd = None

    skew_rpd_data.append({
        'file_key': file_key,
        'f49': skew_rpd
    })

# Create DataFrame 
skew_rpd_df = pd.DataFrame(skew_rpd_data)

feature_df = feature_df.merge(skew_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.638328,108.547275,7.03303,24.56396,22.308431,22.293014,22.294505,0.720713,0.344680,-7.535151
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,3.845505,-1.347748,11.05365,25.73604,21.549576,21.485345,21.603905,1.663640,1.242880,0.188930
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.679110,2.944222,17.58587,23.34963,21.520051,21.483555,21.861390,1.212035,0.500950,-1.630539
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.950445,-0.852725,19.73306,24.60662,22.026777,22.012955,21.909645,0.787555,0.371335,0.862450


In [61]:
#IQR of right pupil diameter
iqr_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        iqr_rpd = iqr(rpd)
    else:
        median_rpd = None

    iqr_rpd_data.append({
        'file_key': file_key,
        'f50': iqr_rpd
    })

# Create DataFrame 
iqr_rpd_df = pd.DataFrame(iqr_rpd_data)

feature_df = feature_df.merge(iqr_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453,2.875732
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182,1.408065
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767,2.183140
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961,2.123840
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731,1.534745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,108.547275,7.03303,24.56396,22.308431,22.293014,22.294505,0.720713,0.344680,-7.535151,0.697615
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,-1.347748,11.05365,25.73604,21.549576,21.485345,21.603905,1.663640,1.242880,0.188930,2.418410
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,2.944222,17.58587,23.34963,21.520051,21.483555,21.861390,1.212035,0.500950,-1.630539,1.024970
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,-0.852725,19.73306,24.60662,22.026777,22.012955,21.909645,0.787555,0.371335,0.862450,0.738805


In [62]:
#Kurtosis of right pupil diameter
kurt_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        kurt_rpd = kurtosis(rpd)
    else:
        median_rpd = None

    kurt_rpd_data.append({
        'file_key': file_key,
        'f51': kurt_rpd
    })

# Create DataFrame 
kurt_rpd_df = pd.DataFrame(kurt_rpd_data)

feature_df = feature_df.merge(kurt_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453,2.875732,-1.121120
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182,1.408065,2.610415
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767,2.183140,-0.324120
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961,2.123840,-0.584666
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731,1.534745,-0.680287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,7.03303,24.56396,22.308431,22.293014,22.294505,0.720713,0.344680,-7.535151,0.697615,142.381072
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,11.05365,25.73604,21.549576,21.485345,21.603905,1.663640,1.242880,0.188930,2.418410,0.346749
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,17.58587,23.34963,21.520051,21.483555,21.861390,1.212035,0.500950,-1.630539,1.024970,2.003091
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,19.73306,24.60662,22.026777,22.012955,21.909645,0.787555,0.371335,0.862450,0.738805,1.234169


In [63]:
#List of all valid(LPV = 1 and RPV = 1) ratio between left pupil and right pupil diameter for each file
ratio_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT LPD/RPD as ratio 
        FROM df
        WHERE LPV = '1' and RPV = '1'
    """).to_df()

    # Extract vectors
    ratio_list = result['ratio'].tolist()

    # Append result 
    ratio_vector.append({
        'file_key': key,
        'ratio': ratio_list
    })

# Create the DataFrame
ratio_vector_df = pd.DataFrame(ratio_vector)

# Display the DataFrame
display(ratio_vector_df)


,file_key,ratio
0,T1_S1_TRY1_VB_SL_LIT,"[1.060344638095293, 1.0229640831418725, 1.0730..."
1,T1_S1_TRY1_HS_SL_LIT,"[1.0464576982861296, 1.044504862714504, 1.0345..."
2,T1_S1_TRY1_VB_SL_BIG,"[1.0602436337841066, 1.0580399491109065, 1.064..."
3,T1_S1_TRY1_HS_SL_BIG,"[1.00851129541736, 1.0032453905083867, 1.00725..."
4,T1_S1_TRY1_VB_FA_LIT,"[1.0550391662418483, 1.0604205395745907, 1.010..."
...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[1.114892921395265, 1.1392036087881785, 1.1331..."
2372,T34_S1_TRY3_VB_FA_LIT,"[1.1573272643798556, 1.163547835366459, 1.1704..."
2373,T34_S1_TRY3_HS_FA_LIT,"[1.0425742173185206, 1.0630686621062184, 1.060..."
2374,T34_S1_TRY3_VB_FA_BIG,"[1.1581022325559436, 1.1750977559328304, 1.163..."


In [64]:
#Minimum ratio bewtween left and right pupil diameter
min_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        min_ratio = np.min(ratio)
    else:
        min_ratio = None

    min_ratio_data.append({
        'file_key': file_key,
        'f52': min_ratio
    })

# Create DataFrame from min ratio
min_ratio_df = pd.DataFrame(min_ratio_data)

feature_df = feature_df.merge(min_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453,2.875732,-1.121120,0.822074
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182,1.408065,2.610415,0.551956
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767,2.183140,-0.324120,0.745355
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961,2.123840,-0.584666,0.986002
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731,1.534745,-0.680287,0.927702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,24.56396,22.308431,22.293014,22.294505,0.720713,0.344680,-7.535151,0.697615,142.381072,0.717253
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,25.73604,21.549576,21.485345,21.603905,1.663640,1.242880,0.188930,2.418410,0.346749,0.985429
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,23.34963,21.520051,21.483555,21.861390,1.212035,0.500950,-1.630539,1.024970,2.003091,1.013748
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,24.60662,22.026777,22.012955,21.909645,0.787555,0.371335,0.862450,0.738805,1.234169,0.990155


In [65]:
#Maximum ratio bewtween left and right pupil diameter
max_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        max_ratio = np.max(ratio)
    else:
        max_ratio = None

    max_ratio_data.append({
        'file_key': file_key,
        'f53': max_ratio
    })

# Create DataFrame from max ratio
max_ratio_df = pd.DataFrame(max_ratio_data)

feature_df = feature_df.merge(max_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453,2.875732,-1.121120,0.822074,1.167402
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182,1.408065,2.610415,0.551956,1.187651
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767,2.183140,-0.324120,0.745355,1.170511
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961,2.123840,-0.584666,0.986002,1.197885
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731,1.534745,-0.680287,0.927702,1.132361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,22.308431,22.293014,22.294505,0.720713,0.344680,-7.535151,0.697615,142.381072,0.717253,1.263720
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,21.549576,21.485345,21.603905,1.663640,1.242880,0.188930,2.418410,0.346749,0.985429,1.215086
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,21.520051,21.483555,21.861390,1.212035,0.500950,-1.630539,1.024970,2.003091,1.013748,1.279799
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,22.026777,22.012955,21.909645,0.787555,0.371335,0.862450,0.738805,1.234169,0.990155,1.224127


In [66]:
#Arithmetic mean ratio bewtween left and right pupil diameter
mean_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        mean_ratio = np.mean(ratio)
    else:
        mean_ratio = None

    mean_ratio_data.append({
        'file_key': file_key,
        'f54': mean_ratio
    })

# Create DataFrame from mean ratio
mean_ratio_df = pd.DataFrame(mean_ratio_data)

feature_df = feature_df.merge(mean_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,17.999585,18.060460,1.527503,1.443570,-0.037453,2.875732,-1.121120,0.822074,1.167402,1.044448
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,19.328542,19.203055,1.005368,0.669080,-0.295182,1.408065,2.610415,0.551956,1.187651,1.088283
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,18.844825,19.103850,1.258265,0.864570,-0.398767,2.183140,-0.324120,0.745355,1.170511,1.057753
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,18.807152,18.396135,1.226277,0.647955,0.535961,2.123840,-0.584666,0.986002,1.197885,1.093910
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,18.622358,18.833000,1.058681,0.690400,-0.505731,1.534745,-0.680287,0.927702,1.132361,1.017737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,22.293014,22.294505,0.720713,0.344680,-7.535151,0.697615,142.381072,0.717253,1.263720,1.097335
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,21.485345,21.603905,1.663640,1.242880,0.188930,2.418410,0.346749,0.985429,1.215086,1.094097
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,21.483555,21.861390,1.212035,0.500950,-1.630539,1.024970,2.003091,1.013748,1.279799,1.096271
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,22.012955,21.909645,0.787555,0.371335,0.862450,0.738805,1.234169,0.990155,1.224127,1.091695


In [67]:
#Geometric mean ratio bewtween left and right pupil diameter
gmean_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        gmean_ratio = gmean(ratio)
    else:
        gmean_ratio = None

    gmean_ratio_data.append({
        'file_key': file_key,
        'f55': gmean_ratio
    })

# Create DataFrame from gmean ratio
gmean_ratio_df = pd.DataFrame(gmean_ratio_data)

feature_df = feature_df.merge(gmean_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,18.060460,1.527503,1.443570,-0.037453,2.875732,-1.121120,0.822074,1.167402,1.044448,1.043929
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,19.203055,1.005368,0.669080,-0.295182,1.408065,2.610415,0.551956,1.187651,1.088283,1.087546
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,19.103850,1.258265,0.864570,-0.398767,2.183140,-0.324120,0.745355,1.170511,1.057753,1.057170
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,18.396135,1.226277,0.647955,0.535961,2.123840,-0.584666,0.986002,1.197885,1.093910,1.092908
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,18.833000,1.058681,0.690400,-0.505731,1.534745,-0.680287,0.927702,1.132361,1.017737,1.017284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,22.294505,0.720713,0.344680,-7.535151,0.697615,142.381072,0.717253,1.263720,1.097335,1.097038
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,21.603905,1.663640,1.242880,0.188930,2.418410,0.346749,0.985429,1.215086,1.094097,1.093197
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,21.861390,1.212035,0.500950,-1.630539,1.024970,2.003091,1.013748,1.279799,1.096271,1.095291
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,21.909645,0.787555,0.371335,0.862450,0.738805,1.234169,0.990155,1.224127,1.091695,1.090606


In [68]:
#median ratio bewtween left and right pupil diameter
median_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        median_ratio = np.median(ratio)
    else:
        median_ratio = None

    median_ratio_data.append({
        'file_key': file_key,
        'f56': median_ratio
    })

# Create DataFrame from median ratio
median_ratio_df = pd.DataFrame(median_ratio_data)

feature_df = feature_df.merge(median_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.527503,1.443570,-0.037453,2.875732,-1.121120,0.822074,1.167402,1.044448,1.043929,1.044883
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.005368,0.669080,-0.295182,1.408065,2.610415,0.551956,1.187651,1.088283,1.087546,1.090124
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.258265,0.864570,-0.398767,2.183140,-0.324120,0.745355,1.170511,1.057753,1.057170,1.059002
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,1.226277,0.647955,0.535961,2.123840,-0.584666,0.986002,1.197885,1.093910,1.092908,1.110694
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.058681,0.690400,-0.505731,1.534745,-0.680287,0.927702,1.132361,1.017737,1.017284,1.015277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.720713,0.344680,-7.535151,0.697615,142.381072,0.717253,1.263720,1.097335,1.097038,1.096055
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.663640,1.242880,0.188930,2.418410,0.346749,0.985429,1.215086,1.094097,1.093197,1.091565
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,1.212035,0.500950,-1.630539,1.024970,2.003091,1.013748,1.279799,1.096271,1.095291,1.084072
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.787555,0.371335,0.862450,0.738805,1.234169,0.990155,1.224127,1.091695,1.090606,1.085269


In [69]:
#std ratio bewtween left and right pupil diameter
std_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        std_ratio = np.std(ratio)
    else:
        std_ratio = None

    std_ratio_data.append({
        'file_key': file_key,
        'f57': std_ratio
    })

# Create DataFrame from std ratio
std_ratio_df = pd.DataFrame(std_ratio_data)

feature_df = feature_df.merge(std_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.443570,-0.037453,2.875732,-1.121120,0.822074,1.167402,1.044448,1.043929,1.044883,0.032832
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.669080,-0.295182,1.408065,2.610415,0.551956,1.187651,1.088283,1.087546,1.090124,0.038902
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.864570,-0.398767,2.183140,-0.324120,0.745355,1.170511,1.057753,1.057170,1.059002,0.034962
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.647955,0.535961,2.123840,-0.584666,0.986002,1.197885,1.093910,1.092908,1.110694,0.046493
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.690400,-0.505731,1.534745,-0.680287,0.927702,1.132361,1.017737,1.017284,1.015277,0.030454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.344680,-7.535151,0.697615,142.381072,0.717253,1.263720,1.097335,1.097038,1.096055,0.024631
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.242880,0.188930,2.418410,0.346749,0.985429,1.215086,1.094097,1.093197,1.091565,0.044587
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.500950,-1.630539,1.024970,2.003091,1.013748,1.279799,1.096271,1.095291,1.084072,0.047661
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.371335,0.862450,0.738805,1.234169,0.990155,1.224127,1.091695,1.090606,1.085269,0.048962


In [70]:
#MAD ratio bewtween left and right pupil diameter
mad_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        mad_ratio = median_abs_deviation(ratio, scale=1)
    else:
        mad_ratio = None

    mad_ratio_data.append({
        'file_key': file_key,
        'f58': mad_ratio
    })

# Create DataFrame from mad ratio
mad_ratio_df = pd.DataFrame(mad_ratio_data)

feature_df = feature_df.merge(mad_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,-0.037453,2.875732,-1.121120,0.822074,1.167402,1.044448,1.043929,1.044883,0.032832,0.018985
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,-0.295182,1.408065,2.610415,0.551956,1.187651,1.088283,1.087546,1.090124,0.038902,0.028758
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,-0.398767,2.183140,-0.324120,0.745355,1.170511,1.057753,1.057170,1.059002,0.034962,0.023063
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.535961,2.123840,-0.584666,0.986002,1.197885,1.093910,1.092908,1.110694,0.046493,0.030646
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,-0.505731,1.534745,-0.680287,0.927702,1.132361,1.017737,1.017284,1.015277,0.030454,0.018253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,-7.535151,0.697615,142.381072,0.717253,1.263720,1.097335,1.097038,1.096055,0.024631,0.011433
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.188930,2.418410,0.346749,0.985429,1.215086,1.094097,1.093197,1.091565,0.044587,0.027373
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,-1.630539,1.024970,2.003091,1.013748,1.279799,1.096271,1.095291,1.084072,0.047661,0.014956
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.862450,0.738805,1.234169,0.990155,1.224127,1.091695,1.090606,1.085269,0.048962,0.036965


In [71]:
#Skewness ratio bewtween left and right pupil diameter
skew_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        skew_ratio = skew(ratio)
    else:
        skew_ratio = None

    skew_ratio_data.append({
        'file_key': file_key,
        'f59': skew_ratio
    })

# Create DataFrame from skew ratio
skew_ratio_df = pd.DataFrame(skew_ratio_data)

feature_df = feature_df.merge(skew_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,2.875732,-1.121120,0.822074,1.167402,1.044448,1.043929,1.044883,0.032832,0.018985,-0.152905
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.408065,2.610415,0.551956,1.187651,1.088283,1.087546,1.090124,0.038902,0.028758,-1.636806
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,2.183140,-0.324120,0.745355,1.170511,1.057753,1.057170,1.059002,0.034962,0.023063,-0.279435
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,2.123840,-0.584666,0.986002,1.197885,1.093910,1.092908,1.110694,0.046493,0.030646,-0.473445
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.534745,-0.680287,0.927702,1.132361,1.017737,1.017284,1.015277,0.030454,0.018253,0.269629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.697615,142.381072,0.717253,1.263720,1.097335,1.097038,1.096055,0.024631,0.011433,-3.530984
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,2.418410,0.346749,0.985429,1.215086,1.094097,1.093197,1.091565,0.044587,0.027373,0.399639
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,1.024970,2.003091,1.013748,1.279799,1.096271,1.095291,1.084072,0.047661,0.014956,2.029805
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.738805,1.234169,0.990155,1.224127,1.091695,1.090606,1.085269,0.048962,0.036965,0.349505


In [72]:
#IQR ratio bewtween left and right pupil diameter
iqr_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        iqr_ratio = iqr(ratio)
    else:
        iqr_ratio = None

    iqr_ratio_data.append({
        'file_key': file_key,
        'f60': iqr_ratio
    })

# Create DataFrame from iqr ratio
iqr_ratio_df = pd.DataFrame(iqr_ratio_data)

feature_df = feature_df.merge(iqr_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,-1.121120,0.822074,1.167402,1.044448,1.043929,1.044883,0.032832,0.018985,-0.152905,0.037992
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,2.610415,0.551956,1.187651,1.088283,1.087546,1.090124,0.038902,0.028758,-1.636806,0.057819
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,-0.324120,0.745355,1.170511,1.057753,1.057170,1.059002,0.034962,0.023063,-0.279435,0.046060
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,-0.584666,0.986002,1.197885,1.093910,1.092908,1.110694,0.046493,0.030646,-0.473445,0.081740
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,-0.680287,0.927702,1.132361,1.017737,1.017284,1.015277,0.030454,0.018253,0.269629,0.036962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,142.381072,0.717253,1.263720,1.097335,1.097038,1.096055,0.024631,0.011433,-3.530984,0.022796
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.346749,0.985429,1.215086,1.094097,1.093197,1.091565,0.044587,0.027373,0.399639,0.054956
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,2.003091,1.013748,1.279799,1.096271,1.095291,1.084072,0.047661,0.014956,2.029805,0.030216
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.234169,0.990155,1.224127,1.091695,1.090606,1.085269,0.048962,0.036965,0.349505,0.076137


In [73]:
#kurtosis ratio bewtween left and right pupil diameter
kurt_ratio_data = []

for item in ratio_vector:
    file_key = item['file_key']
    ratio = item['ratio']

    #Check if the list is empty
    if ratio:  
        kurt_ratio = kurtosis(ratio)
    else:
        kurt_ratio = None

    kurt_ratio_data.append({
        'file_key': file_key,
        'f61': kurt_ratio
    })

# Create DataFrame from kurt ratio
kurt_ratio_df = pd.DataFrame(kurt_ratio_data)

feature_df = feature_df.merge(kurt_ratio_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.822074,1.167402,1.044448,1.043929,1.044883,0.032832,0.018985,-0.152905,0.037992,1.227744
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.551956,1.187651,1.088283,1.087546,1.090124,0.038902,0.028758,-1.636806,0.057819,18.075603
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.745355,1.170511,1.057753,1.057170,1.059002,0.034962,0.023063,-0.279435,0.046060,2.400227
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.986002,1.197885,1.093910,1.092908,1.110694,0.046493,0.030646,-0.473445,0.081740,-1.049548
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.927702,1.132361,1.017737,1.017284,1.015277,0.030454,0.018253,0.269629,0.036962,0.544264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.717253,1.263720,1.097335,1.097038,1.096055,0.024631,0.011433,-3.530984,0.022796,56.420118
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.985429,1.215086,1.094097,1.093197,1.091565,0.044587,0.027373,0.399639,0.054956,-0.402821
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,1.013748,1.279799,1.096271,1.095291,1.084072,0.047661,0.014956,2.029805,0.030216,3.709015
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.990155,1.224127,1.091695,1.090606,1.085269,0.048962,0.036965,0.349505,0.076137,-0.868745


In [74]:
#List of all valid(LPV = 1 and RPV = 1) difference between left pupil and right pupil diameter for each file
diff_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT (LPD - RPD) as diff
        FROM df
        WHERE LPV = '1' and RPV = '1'
    """).to_df()

    # Extract vectors
    diff_list = result['diff'].tolist()

    # Append result 
    diff_vector.append({
        'file_key': key,
        'diff': diff_list
    })

# Create the DataFrame
diff_vector_df = pd.DataFrame(diff_vector)

# Display the DataFrame
display(diff_vector_df)


,file_key,diff
0,T1_S1_TRY1_VB_SL_LIT,"[1.043219999999998, 0.40940000000000154, 1.378..."
1,T1_S1_TRY1_HS_SL_LIT,"[0.9205499999999986, 0.8739500000000007, 0.687..."
2,T1_S1_TRY1_VB_SL_BIG,"[1.014009999999999, 0.9899699999999996, 1.0989..."
3,T1_S1_TRY1_HS_SL_BIG,"[0.14165000000000205, 0.05977000000000032, 0.1..."
4,T1_S1_TRY1_VB_FA_LIT,"[1.0505099999999992, 1.1414600000000021, 0.201..."
...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[2.6285799999999995, 3.1580900000000014, 3.039..."
2372,T34_S1_TRY3_VB_FA_LIT,"[3.6769599999999976, 3.788529999999998, 3.8069..."
2373,T34_S1_TRY3_HS_FA_LIT,"[0.9038600000000017, 1.3385099999999994, 1.264..."
2374,T34_S1_TRY3_VB_FA_BIG,"[3.4155599999999993, 3.7498200000000033, 3.518..."


In [ ]:
#Geometric mean difference bewtween left and right pupil diameter
gmean_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        gmean_diff = np.mean(diff)
    else:
        mean_diff = None

    mean_diff_data.append({
        'file_key': file_key,
        'f64': mean_diff
    })

# Create DataFrame from mean diff
mean_diff_df = pd.DataFrame(mean_diff_data)

feature_df = feature_df.merge(mean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [75]:
#Minimum difference bewtween left and right pupil diameter
min_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        min_diff = np.min(diff)
    else:
        min_diff = None

    min_diff_data.append({
        'file_key': file_key,
        'f62': min_diff
    })

# Create DataFrame from min diff
min_diff_df = pd.DataFrame(min_diff_data)

feature_df = feature_df.merge(min_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.167402,1.044448,1.043929,1.044883,0.032832,0.018985,-0.152905,0.037992,1.227744,-3.55531
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.187651,1.088283,1.087546,1.090124,0.038902,0.028758,-1.636806,0.057819,18.075603,-7.39138
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.170511,1.057753,1.057170,1.059002,0.034962,0.023063,-0.279435,0.046060,2.400227,-4.85369
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,1.197885,1.093910,1.092908,1.110694,0.046493,0.030646,-0.473445,0.081740,-1.049548,-0.29613
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.132361,1.017737,1.017284,1.015277,0.030454,0.018253,0.269629,0.036962,0.544264,-1.37072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,1.263720,1.097335,1.097038,1.096055,0.024631,0.011433,-3.530984,0.022796,56.420118,-4.92811
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.215086,1.094097,1.093197,1.091565,0.044587,0.027373,0.399639,0.054956,-0.402821,-0.30719
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,1.279799,1.096271,1.095291,1.084072,0.047661,0.014956,2.029805,0.030216,3.709015,0.28851
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.224127,1.091695,1.090606,1.085269,0.048962,0.036965,0.349505,0.076137,-0.868745,-0.22123


In [76]:
#Maximum difference bewtween left and right pupil diameter
max_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        max_diff = np.max(diff)
    else:
        max_diff = None

    max_diff_data.append({
        'file_key': file_key,
        'f63': max_diff
    })

# Create DataFrame from max diff
max_diff_df = pd.DataFrame(max_diff_data)

feature_df = feature_df.merge(max_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.044448,1.043929,1.044883,0.032832,0.018985,-0.152905,0.037992,1.227744,-3.55531,3.12990
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.088283,1.087546,1.090124,0.038902,0.028758,-1.636806,0.057819,18.075603,-7.39138,3.44789
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.057753,1.057170,1.059002,0.034962,0.023063,-0.279435,0.046060,2.400227,-4.85369,2.70695
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,1.093910,1.092908,1.110694,0.046493,0.030646,-0.473445,0.081740,-1.049548,-0.29613,3.46026
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.017737,1.017284,1.015277,0.030454,0.018253,0.269629,0.036962,0.544264,-1.37072,2.19886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,1.097335,1.097038,1.096055,0.024631,0.011433,-3.530984,0.022796,56.420118,-4.92811,5.00648
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.094097,1.093197,1.091565,0.044587,0.027373,0.399639,0.054956,-0.402821,-0.30719,4.53475
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,1.096271,1.095291,1.084072,0.047661,0.014956,2.029805,0.030216,3.709015,0.28851,5.09679
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.091695,1.090606,1.085269,0.048962,0.036965,0.349505,0.076137,-0.868745,-0.22123,4.53786


In [77]:
#Arithmetic mean difference bewtween left and right pupil diameter
mean_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        mean_diff = np.mean(diff)
    else:
        mean_diff = None

    mean_diff_data.append({
        'file_key': file_key,
        'f64': mean_diff
    })

# Create DataFrame from mean diff
mean_diff_df = pd.DataFrame(mean_diff_data)

feature_df = feature_df.merge(mean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.043929,1.044883,0.032832,0.018985,-0.152905,0.037992,1.227744,-3.55531,3.12990,0.815898
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.087546,1.090124,0.038902,0.028758,-1.636806,0.057819,18.075603,-7.39138,3.44789,1.687242
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.057170,1.059002,0.034962,0.023063,-0.279435,0.046060,2.400227,-4.85369,2.70695,1.087334
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,1.092908,1.110694,0.046493,0.030646,-0.473445,0.081740,-1.049548,-0.29613,3.46026,1.723565
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.017284,1.015277,0.030454,0.018253,0.269629,0.036962,0.544264,-1.37072,2.19886,0.320529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,1.097038,1.096055,0.024631,0.011433,-3.530984,0.022796,56.420118,-4.92811,5.00648,2.173023
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.093197,1.091565,0.044587,0.027373,0.399639,0.054956,-0.402821,-0.30719,4.53475,2.031564
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,1.095291,1.084072,0.047661,0.014956,2.029805,0.030216,3.709015,0.28851,5.09679,2.021018
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.090606,1.085269,0.048962,0.036965,0.349505,0.076137,-0.868745,-0.22123,4.53786,2.011972


In [78]:
#median difference bewtween left and right pupil diameter
median_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        median_diff = np.median(diff)
    else:
        median_diff = None

    median_diff_data.append({
        'file_key': file_key,
        'f65': median_diff
    })

# Create DataFrame from median diff
median_diff_df = pd.DataFrame(median_diff_data)

feature_df = feature_df.merge(median_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.044883,0.032832,0.018985,-0.152905,0.037992,1.227744,-3.55531,3.12990,0.815898,0.826380
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,1.090124,0.038902,0.028758,-1.636806,0.057819,18.075603,-7.39138,3.44789,1.687242,1.740425
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,1.059002,0.034962,0.023063,-0.279435,0.046060,2.400227,-4.85369,2.70695,1.087334,1.124960
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,1.110694,0.046493,0.030646,-0.473445,0.081740,-1.049548,-0.29613,3.46026,1.723565,2.022720
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,1.015277,0.030454,0.018253,0.269629,0.036962,0.544264,-1.37072,2.19886,0.320529,0.283560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,1.096055,0.024631,0.011433,-3.530984,0.022796,56.420118,-4.92811,5.00648,2.173023,2.138820
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,1.091565,0.044587,0.027373,0.399639,0.054956,-0.402821,-0.30719,4.53475,2.031564,1.956255
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,1.084072,0.047661,0.014956,2.029805,0.030216,3.709015,0.28851,5.09679,2.021018,1.837410
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,1.085269,0.048962,0.036965,0.349505,0.076137,-0.868745,-0.22123,4.53786,2.011972,1.866455


In [79]:
#std difference bewtween left and right pupil diameter
std_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        std_diff = np.std(diff)
    else:
        std_diff = None

    std_diff_data.append({
        'file_key': file_key,
        'f66': std_diff
    })

# Create DataFrame from std diff
std_diff_df = pd.DataFrame(std_diff_data)

feature_df = feature_df.merge(std_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.032832,0.018985,-0.152905,0.037992,1.227744,-3.55531,3.12990,0.815898,0.826380,0.604321
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.038902,0.028758,-1.636806,0.057819,18.075603,-7.39138,3.44789,1.687242,1.740425,0.690667
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.034962,0.023063,-0.279435,0.046060,2.400227,-4.85369,2.70695,1.087334,1.124960,0.647141
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.046493,0.030646,-0.473445,0.081740,-1.049548,-0.29613,3.46026,1.723565,2.022720,0.805627
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.030454,0.018253,0.269629,0.036962,0.544264,-1.37072,2.19886,0.320529,0.283560,0.555955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.024631,0.011433,-3.530984,0.022796,56.420118,-4.92811,5.00648,2.173023,2.138820,0.507144
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.044587,0.027373,0.399639,0.054956,-0.402821,-0.30719,4.53475,2.031564,1.956255,0.995214
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.047661,0.014956,2.029805,0.030216,3.709015,0.28851,5.09679,2.021018,1.837410,0.826467
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.048962,0.036965,0.349505,0.076137,-0.868745,-0.22123,4.53786,2.011972,1.866455,1.067011


In [80]:
#MAD difference bewtween left and right pupil diameter
mad_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        mad_diff = median_abs_deviation(diff, scale=1)
    else:
        mad_diff = None

    mad_diff_data.append({
        'file_key': file_key,
        'f67': mad_diff
    })

# Create DataFrame from mad diff
mad_diff_df = pd.DataFrame(mad_diff_data)

feature_df = feature_df.merge(mad_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.018985,-0.152905,0.037992,1.227744,-3.55531,3.12990,0.815898,0.826380,0.604321,0.351930
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.028758,-1.636806,0.057819,18.075603,-7.39138,3.44789,1.687242,1.740425,0.690667,0.504960
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.023063,-0.279435,0.046060,2.400227,-4.85369,2.70695,1.087334,1.124960,0.647141,0.452050
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.030646,-0.473445,0.081740,-1.049548,-0.29613,3.46026,1.723565,2.022720,0.805627,0.504900
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.018253,0.269629,0.036962,0.544264,-1.37072,2.19886,0.320529,0.283560,0.555955,0.344610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.011433,-3.530984,0.022796,56.420118,-4.92811,5.00648,2.173023,2.138820,0.507144,0.237420
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.027373,0.399639,0.054956,-0.402821,-0.30719,4.53475,2.031564,1.956255,0.995214,0.645070
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.014956,2.029805,0.030216,3.709015,0.28851,5.09679,2.021018,1.837410,0.826467,0.302480
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.036965,0.349505,0.076137,-0.868745,-0.22123,4.53786,2.011972,1.866455,1.067011,0.791470


In [81]:
#Skewness difference bewtween left and right pupil diameter
skew_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        skew_diff = skew(diff)
    else:
        skew_diff = None

    skew_diff_data.append({
        'file_key': file_key,
        'f68': skew_diff
    })

# Create DataFrame from skew diff
skew_diff_df = pd.DataFrame(skew_diff_data)

feature_df = feature_df.merge(skew_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,-0.152905,0.037992,1.227744,-3.55531,3.12990,0.815898,0.826380,0.604321,0.351930,-0.105286
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,-1.636806,0.057819,18.075603,-7.39138,3.44789,1.687242,1.740425,0.690667,0.504960,-1.604894
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,-0.279435,0.046060,2.400227,-4.85369,2.70695,1.087334,1.124960,0.647141,0.452050,-0.430846
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,-0.473445,0.081740,-1.049548,-0.29613,3.46026,1.723565,2.022720,0.805627,0.504900,-0.555203
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.269629,0.036962,0.544264,-1.37072,2.19886,0.320529,0.283560,0.555955,0.344610,0.097320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,-3.530984,0.022796,56.420118,-4.92811,5.00648,2.173023,2.138820,0.507144,0.237420,-2.338623
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.399639,0.054956,-0.402821,-0.30719,4.53475,2.031564,1.956255,0.995214,0.645070,0.495952
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,2.029805,0.030216,3.709015,0.28851,5.09679,2.021018,1.837410,0.826467,0.302480,1.817304
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.349505,0.076137,-0.868745,-0.22123,4.53786,2.011972,1.866455,1.067011,0.791470,0.335808


In [82]:
#IQR difference bewtween left and right pupil diameter
iqr_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        iqr_diff = iqr(diff)
    else:
        iqr_diff = None

    iqr_diff_data.append({
        'file_key': file_key,
        'f69': iqr_diff
    })

# Create DataFrame from iqr diff
iqr_diff_df = pd.DataFrame(iqr_diff_data)

feature_df = feature_df.merge(iqr_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,0.037992,1.227744,-3.55531,3.12990,0.815898,0.826380,0.604321,0.351930,-0.105286,0.707010
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,0.057819,18.075603,-7.39138,3.44789,1.687242,1.740425,0.690667,0.504960,-1.604894,1.000752
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,0.046060,2.400227,-4.85369,2.70695,1.087334,1.124960,0.647141,0.452050,-0.430846,0.902730
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,0.081740,-1.049548,-0.29613,3.46026,1.723565,2.022720,0.805627,0.504900,-0.555203,1.371867
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.036962,0.544264,-1.37072,2.19886,0.320529,0.283560,0.555955,0.344610,0.097320,0.696155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,0.022796,56.420118,-4.92811,5.00648,2.173023,2.138820,0.507144,0.237420,-2.338623,0.476560
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,0.054956,-0.402821,-0.30719,4.53475,2.031564,1.956255,0.995214,0.645070,0.495952,1.280085
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.030216,3.709015,0.28851,5.09679,2.021018,1.837410,0.826467,0.302480,1.817304,0.602227
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,0.076137,-0.868745,-0.22123,4.53786,2.011972,1.866455,1.067011,0.791470,0.335808,1.636978


In [83]:
#kurtosis difference bewtween left and right pupil diameter
kurt_diff_data = []

for item in diff_vector:
    file_key = item['file_key']
    diff = item['diff']

    #Check if the list is empty
    if diff:  
        kurt_diff = kurtosis(diff)
    else:
        kurt_diff = None

    kurt_diff_data.append({
        'file_key': file_key,
        'f70': kurt_diff
    })

# Create DataFrame from kurt diff
kurt_diff_df = pd.DataFrame(kurt_diff_data)

feature_df = feature_df.merge(kurt_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,1.227744,-3.55531,3.12990,0.815898,0.826380,0.604321,0.351930,-0.105286,0.707010,1.461003
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,18.075603,-7.39138,3.44789,1.687242,1.740425,0.690667,0.504960,-1.604894,1.000752,15.569021
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,2.400227,-4.85369,2.70695,1.087334,1.124960,0.647141,0.452050,-0.430846,0.902730,2.502309
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,-1.049548,-0.29613,3.46026,1.723565,2.022720,0.805627,0.504900,-0.555203,1.371867,-0.896456
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,0.544264,-1.37072,2.19886,0.320529,0.283560,0.555955,0.344610,0.097320,0.696155,0.343982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,56.420118,-4.92811,5.00648,2.173023,2.138820,0.507144,0.237420,-2.338623,0.476560,33.248300
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,-0.402821,-0.30719,4.53475,2.031564,1.956255,0.995214,0.645070,0.495952,1.280085,-0.408691
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,3.709015,0.28851,5.09679,2.021018,1.837410,0.826467,0.302480,1.817304,0.602227,3.304574
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,-0.868745,-0.22123,4.53786,2.011972,1.866455,1.067011,0.791470,0.335808,1.636978,-0.919747


In [84]:
#Number of blinks
#Obtained counting the number of BKID different from zero in each file

bkid_counts = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    result = duckdb.query("""
        SELECT COUNT(DISTINCT BKID) AS BKID_COUNT
        FROM df
        WHERE CAST(BKID AS INTEGER) > 0
    """).to_df()

    # Append result with file key and count
    bkid_counts.append({
        'file_key': key,
        'f71': result['BKID_COUNT'][0]
    })

# Convert list of dicts to a DataFrame
bkid_counts_df = pd.DataFrame(bkid_counts)

feature_df = feature_df.merge(bkid_counts_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,-3.55531,3.12990,0.815898,0.826380,0.604321,0.351930,-0.105286,0.707010,1.461003,1
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,-7.39138,3.44789,1.687242,1.740425,0.690667,0.504960,-1.604894,1.000752,15.569021,3
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,-4.85369,2.70695,1.087334,1.124960,0.647141,0.452050,-0.430846,0.902730,2.502309,1
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,-0.29613,3.46026,1.723565,2.022720,0.805627,0.504900,-0.555203,1.371867,-0.896456,2
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,-1.37072,2.19886,0.320529,0.283560,0.555955,0.344610,0.097320,0.696155,0.343982,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,-4.92811,5.00648,2.173023,2.138820,0.507144,0.237420,-2.338623,0.476560,33.248300,2
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,-0.30719,4.53475,2.031564,1.956255,0.995214,0.645070,0.495952,1.280085,-0.408691,1
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,0.28851,5.09679,2.021018,1.837410,0.826467,0.302480,1.817304,0.602227,3.304574,0
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,-0.22123,4.53786,2.011972,1.866455,1.067011,0.791470,0.335808,1.636978,-0.919747,0


In [92]:
#Blink duration list
#Obtained selecting all the BKDUR different from zero

bkdur_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    result = duckdb.query("""
        SELECT BKDUR
        FROM df
        WHERE CAST(BKDUR AS DOUBLE) > 0.0
    """).to_df()

    bkdur_list = result['BKDUR'].tolist()


    # Append result with file key and count
    bkdur_vector.append({
        'file_key': key,
        'BKDUR': bkdur_list
    })

# Convert list of dicts to a DataFrame
bkdur_vector_df = pd.DataFrame(bkdur_vector)

display(bkdur_vector_df)


,file_key,BKDUR
0,T1_S1_TRY1_VB_SL_LIT,[0.18165]
1,T1_S1_TRY1_HS_SL_LIT,"[0.17533, 0.12931, 0.13774]"
2,T1_S1_TRY1_VB_SL_BIG,[0.18513]
3,T1_S1_TRY1_HS_SL_BIG,"[0.15181, 0.21207]"
4,T1_S1_TRY1_VB_FA_LIT,[]
...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,"[0.08313, 0.09361]"
2372,T34_S1_TRY3_VB_FA_LIT,[0.23594]
2373,T34_S1_TRY3_HS_FA_LIT,[]
2374,T34_S1_TRY3_VB_FA_BIG,[]


In [97]:
#Arithmetic mean over all the BKDUR different from zero

mean_bkdur = []

for item in bkdur_vector:
    file_key = item['file_key']
    bkdur = item['BKDUR']

    #Check if the list is empty
    if bkdur:  
        mean = np.mean(bkdur)
    else:
        mean = None

    mean_bkdur.append({
        'file_key': file_key,
        'f72': mean
    })

# Create DataFrame from mean diff
mean_bkdur_df = pd.DataFrame(mean_bkdur)

feature_df = feature_df.merge(mean_bkdur_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.24658,0.108956,0.071780,0.900959,...,3.12990,0.815898,0.826380,0.604321,0.351930,-0.105286,0.707010,1.461003,1,0.181650
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.26721,0.172174,0.099730,0.966898,...,3.44789,1.687242,1.740425,0.690667,0.504960,-1.604894,1.000752,15.569021,3,0.147460
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.22656,0.128417,0.041380,3.328440,...,2.70695,1.087334,1.124960,0.647141,0.452050,-0.430846,0.902730,2.502309,1,0.185130
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154105,0.132798,0.12073,0.106873,0.032840,3.084167,...,3.46026,1.723565,2.022720,0.805627,0.504900,-0.555203,1.371867,-0.896456,2,0.181940
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.239979,0.207705,0.22119,0.159702,0.063050,4.316584,...,2.19886,0.320529,0.283560,0.555955,0.344610,0.097320,0.696155,0.343982,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,T34_S1_TRY3_HS_SL_BIG,48,0.10736,1.84009,0.327239,0.265933,0.23749,0.277464,0.092865,3.524131,...,5.00648,2.173023,2.138820,0.507144,0.237420,-2.338623,0.476560,33.248300,2,0.088370
2372,T34_S1_TRY3_VB_FA_LIT,71,0.02002,0.53595,0.198023,0.179392,0.19821,0.083024,0.057490,0.885060,...,4.53475,2.031564,1.956255,0.995214,0.645070,0.495952,1.280085,-0.408691,1,0.235940
2373,T34_S1_TRY3_HS_FA_LIT,58,0.10730,1.40656,0.269362,0.229498,0.20761,0.195011,0.066955,3.538925,...,5.09679,2.021018,1.837410,0.826467,0.302480,1.817304,0.602227,3.304574,0,NaN
2374,T34_S1_TRY3_VB_FA_BIG,78,0.05292,0.82410,0.174235,0.156477,0.16089,0.096828,0.060150,3.816048,...,4.53786,2.011972,1.866455,1.067011,0.791470,0.335808,1.636978,-0.919747,0,NaN
